# models behavior vs human behavior all participants

In [1]:
# -------------------------------------------------
# Core imports
import os
import math
import pathlib
import warnings
from collections import defaultdict
from pathlib import Path

# -------------------------------------------------
# Data & computation
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit, OptimizeWarning
from scipy.spatial.distance import euclidean, correlation, squareform
from scipy.cluster.hierarchy import linkage, leaves_list
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

# -------------------------------------------------
# Statistics
from scipy.stats import (
    pearsonr, spearmanr, sem, norm, f_oneway, ttest_ind
)
from statsmodels.stats.multitest import fdrcorrection, multipletests
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm
from statsmodels.stats.sandwich_covariance import cov_cluster
from statsmodels.genmod.bayes_mixed_glm import BinomialBayesMixedGLM

# -------------------------------------------------
# Plotting
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import MaxNLocator, FixedLocator
from matplotlib.patches import Patch
from matplotlib.backends.backend_pdf import PdfPages
import ast

# -------------------------------------------------
# Settings
matplotlib.use('svg')
matplotlib.rcParams['svg.fonttype'] = 'none'
os.environ["OMP_NUM_THREADS"] = "1"
warnings.filterwarnings("ignore", category=OptimizeWarning)
warnings.filterwarnings("ignore")

# -------------------------------------------------
# Task-specific maps
distributions_map = {"uniform": 0, "low": 1, "high": 2}
actions = {"arrowdown": 0, "arrowup": 1}


# healthy data reading

In [2]:
output_dir = r"29_RL_agent_TD_learn_models_behavior_across_models"
os.makedirs(output_dir, exist_ok=True)


folder_path_participants = 'data_risk_added'

folder_path_greedy = 'healthy/13_RL_agent_TDlearn_output/model_behavior'
folder_path_softmax = 'healthy/13_RL_agent_TDlearn_output_softmax/model_behavior'
folder_path_dualQ = 'healthy/13_RL_agent_TDlearn_output_risk_dualQ/model_behavior'
folder_path_rs = 'healthy/13_RL_agent_TDlearn_output_risk_sensitive/model_behavior'
folder_path_wsls = 'healthy/13_RL_agent_TDlearn_output_wsls/model_behavior'


df_participants = []
df_greedy = []
df_softmax = []
df_dualQ = []
df_rs = []
df_wsls = []



def find_matching_csv(folder_path, df_list):
            for csv_file in os.listdir(folder_path):
                if clean_name in csv_file and csv_file.endswith('.csv'):
                    csv_path = os.path.join(folder_path, csv_file)
                    df_csv = pd.read_csv(csv_path)
                    df_list.append(df_csv)





for file_name in os.listdir(folder_path_participants):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path_participants, file_name)
        df = pd.read_excel(file_path)
        df = df[df['outcome'].str.lower() != 'na'].reset_index(drop=True) 
        df_participants.append(df)

        clean_name = file_name.removeprefix("task_data_").removesuffix(".xlsx")


        find_matching_csv(folder_path_greedy, df_greedy)
        find_matching_csv(folder_path_softmax, df_softmax)
        find_matching_csv(folder_path_dualQ, df_dualQ)
        find_matching_csv(folder_path_rs, df_rs)
        find_matching_csv(folder_path_wsls, df_wsls)

# epileptic data reading

In [3]:
folder_path_participants = 'data_risk_added_epileptic'

folder_path_greedy = 'epileptic/13_RL_agent_TDlearn_output/model_behavior'
folder_path_softmax = 'epileptic/13_RL_agent_TDlearn_output_softmax/model_behavior'
folder_path_dualQ = 'epileptic/13_RL_agent_TDlearn_output_risk_dualQ/model_behavior'
folder_path_rs = 'epileptic/13_RL_agent_TDlearn_output_risk_sensitive/model_behavior'
folder_path_wsls = 'epileptic/13_RL_agent_TDlearn_output_wsls/model_behavior'


df_participants_ep = []
df_greedy_ep = []
df_softmax_ep = []
df_dualQ_ep = []
df_rs_ep = []
df_wsls_ep = []



def find_matching_csv(folder_path, df_list):
            for csv_file in os.listdir(folder_path):
                if clean_name in csv_file and csv_file.endswith('.csv'):
                    csv_path = os.path.join(folder_path, csv_file)
                    df_csv = pd.read_csv(csv_path)
                    df_list.append(df_csv)





for file_name in os.listdir(folder_path_participants):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path_participants, file_name)
        df = pd.read_csv(file_path)
        df = df[df['outcome'].str.lower() != 'na'].reset_index(drop=True) 
        df_participants_ep.append(df)

        clean_name = file_name.removeprefix("task_data_").removesuffix(".csv")


        find_matching_csv(folder_path_greedy, df_greedy_ep)
        find_matching_csv(folder_path_softmax, df_softmax_ep)
        find_matching_csv(folder_path_dualQ, df_dualQ_ep)
        find_matching_csv(folder_path_rs, df_rs_ep)
        find_matching_csv(folder_path_wsls, df_wsls_ep)

# combining both

In [4]:
df_participants = df_participants +df_participants_ep
df_greedy = df_greedy+ df_greedy_ep
df_softmax = df_softmax+ df_softmax_ep
df_dualQ = df_dualQ+ df_dualQ_ep
df_rs = df_rs+ df_rs_ep
df_wsls = df_wsls+df_wsls_ep

num_participants = len(df_participants)
num_participants

47

In [5]:
for i in range(len(df_participants)):
    block = df_participants[i]['block']
    
    for df_list in [df_participants, df_wsls, df_greedy, df_softmax, df_dualQ, df_rs]:
        df_list[i]['block'] = block

In [6]:
print("number of participants:" ,len(df_participants))

number of participants: 47


In [7]:
for df in df_participants:
    df['block_type'] = None
    df.loc[df['block'] == 4, 'block_type'] = 'mix'              # block 4 is mix
    df.loc[df['block'].isin([1, 2, 3]), 'block_type'] = 'fix'   # else is fix

    # Convert totalReward column to float
    df['totalReward'] = df['totalReward'].astype(float)
    df['arrowRT'] = df['arrowRT'].astype(float)


    

for df in df_greedy:
    df['block_type'] = None
    df.loc[df['block'] == 4, 'block_type'] = 'mix'              # block 4 is mix
    df.loc[df['block'].isin([1, 2, 3]), 'block_type'] = 'fix'   # else is fix   

for df in df_softmax:
    df['block_type'] = None
    df.loc[df['block'] == 4, 'block_type'] = 'mix'              # block 4 is mix
    df.loc[df['block'].isin([1, 2, 3]), 'block_type'] = 'fix'   # else is fix


for df in df_dualQ:
    df['block_type'] = None
    df.loc[df['block'] == 4, 'block_type'] = 'mix'              # block 4 is mix
    df.loc[df['block'].isin([1, 2, 3]), 'block_type'] = 'fix'   # else is fix

for df in df_rs:
    df['block_type'] = None
    df.loc[df['block'] == 4, 'block_type'] = 'mix'              # block 4 is mix
    df.loc[df['block'].isin([1, 2, 3]), 'block_type'] = 'fix'   # else is fix

for df in df_wsls:
    df['block_type'] = None
    df.loc[df['block'] == 4, 'block_type'] = 'mix'              # block 4 is mix
    df.loc[df['block'].isin([1, 2, 3]), 'block_type'] = 'fix'   # else is fix




# adding myCard and youCard to the models dataset and also adding model_outcome

In [8]:
for i in range(len(df_participants)):
    myCard = df_participants[i]['myCard']
    yourCard = df_participants[i]['yourCard']
    distributions = df_participants[i]['distribution']
    block_type = df_participants[i]['block_type']
    
    for df_list in [df_participants, df_wsls, df_greedy, df_softmax, df_dualQ, df_rs]:
        df_list[i]['myCard'] = myCard
        df_list[i]['yourCard'] = yourCard
        df_list[i]['distribution'] = distributions
        df_list[i]['block_type'] = block_type

# adding model outcome

In [9]:
for df in df_wsls:
    outcomes = []
    for i in range(len(df)):
        my = df.loc[i, 'myCard']
        your = df.loc[i, 'yourCard']
        choice = df.loc[i, 'model_choices']
        
        if ((my > your and choice == 1) or (my < your and choice == 0)):
            outcomes.append('win')
        else:
            outcomes.append('lose')
    
    df['outcome'] = outcomes



for df in df_greedy:
    outcomes = []
    for i in range(len(df)):
        my = df.loc[i, 'myCard']
        your = df.loc[i, 'yourCard']
        choice = df.loc[i, 'model_choices']
        
        if ((my > your and choice == 1) or (my < your and choice == 0)):
            outcomes.append('win')
        else:
            outcomes.append('lose')
    
    df['outcome'] = outcomes
    
    


for df in df_softmax:
    outcomes = []
    for i in range(len(df)):
        my = df.loc[i, 'myCard']
        your = df.loc[i, 'yourCard']
        choice = df.loc[i, 'model_choices']
        
        if ((my > your and choice == 1) or (my < your and choice == 0)):
            outcomes.append('win')
        else:
            outcomes.append('lose')
    
    df['outcome'] = outcomes
    
    

for df in df_dualQ:
    outcomes = []
    for i in range(len(df)):
        my = df.loc[i, 'myCard']
        your = df.loc[i, 'yourCard']
        choice = df.loc[i, 'model_choices']
        
        if ((my > your and choice == 1) or (my < your and choice == 0)):
            outcomes.append('win')
        else:
            outcomes.append('lose')
    
    df['outcome'] = outcomes


for df in df_rs:
    outcomes = []
    for i in range(len(df)):
        my = df.loc[i, 'myCard']
        your = df.loc[i, 'yourCard']
        choice = df.loc[i, 'model_choices']
        
        if ((my > your and choice == 1) or (my < your and choice == 0)):
            outcomes.append('win')
        else:
            outcomes.append('lose')
    
    df['outcome'] = outcomes

In [10]:
df_participants[2]

,arrowRT,distribution,interTrialInterval,outcome,myCard,yourCard,spaceRT,totalReward,trialIndex,trialType,choice,block,timeoutRepeat,is_within_IQR,risk,block_type
0,2609.0,uniform,789,lose,4,2,335,9.5,0,response,arrowdown,1,0,1,0.375,fix
1,597.0,uniform,853,win,9,4,407,10.0,1,response,arrowup,1,0,1,0.000,fix
2,188.0,uniform,904,win,4,7,504,10.5,2,response,arrowdown,1,0,1,0.375,fix
3,423.0,uniform,916,win,2,4,434,11.0,3,response,arrowdown,1,0,1,0.125,fix
4,549.0,uniform,806,win,5,7,287,11.5,4,response,arrowdown,1,0,1,0.500,fix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,761.0,high,913,win,7,8,382,80.0,125,response,arrowdown,4,0,1,0.447,mix
266,596.0,low,921,win,4,3,318,80.5,83,response,arrowup,4,0,1,0.385,mix
267,414.0,low,950,win,2,7,335,81.0,77,response,arrowdown,4,0,1,0.243,mix
268,1371.0,uniform,842,win,6,4,615,81.5,35,response,arrowup,4,0,1,0.375,mix


In [11]:
df_greedy[2]

,model_choices,participant_choices,model_total_reward,participant_total_reward,q_val,block,block_type,myCard,yourCard,distribution,outcome
0,1,0,10.5,9.5,"[[[0.022019422455794066, 0.02059341150055373],...",1,fix,4,2,uniform,win
1,1,1,11.0,10.0,"[[[0.022019422455794066, 0.02059341150055373],...",1,fix,9,4,uniform,win
2,0,0,11.5,10.5,"[[[0.022019422455794066, 0.02059341150055373],...",1,fix,4,7,uniform,win
3,0,0,12.0,11.0,"[[[0.022019422455794066, 0.02059341150055373],...",1,fix,2,4,uniform,win
4,1,0,11.5,11.5,"[[[0.022019422455794066, 0.02059341150055373],...",1,fix,5,7,uniform,lose
...,...,...,...,...,...,...,...,...,...,...,...
265,0,0,78.0,80.0,"[[[0.4991753037517269, 0.02059341150055373], [...",4,mix,7,8,high,win
266,1,1,78.5,80.5,"[[[0.4991753037517269, 0.02059341150055373], [...",4,mix,4,3,low,win
267,0,0,79.0,81.0,"[[[0.4991753037517269, 0.02059341150055373], [...",4,mix,2,7,low,win
268,1,1,79.5,81.5,"[[[0.4991753037517269, 0.02059341150055373], [...",4,mix,6,4,uniform,win


# total reward

In [12]:
num_subplots = len(df_participants)
trial_num = len(df_participants[0])

fig, axes = plt.subplots(nrows=7, ncols=7, figsize=(15, 18))
axes = axes.flatten()



labels = ['participants','wsls','greedy', 'softmax',  'dualQ', 'RS']
colors = ['black','#E69F00', '#56B4E9', '#009E73', "#CEC10F", "#065B8D"]


for i in range(num_subplots):
    ax = axes[i]

    ax.plot(range(trial_num), (df_participants[i]['totalReward'] - df_participants[i]['totalReward'].iloc[0] )+10, label=labels[0], color=colors[0], linewidth=1.5)
    ax.plot(range(trial_num), df_wsls[i]['model_total_reward'], label=labels[1], color=colors[1], linewidth=0.5)
    ax.plot(range(trial_num), df_greedy[i]['model_total_reward'], label=labels[2], color=colors[2], linewidth=0.5)
    ax.plot(range(trial_num), df_softmax[i]['model_total_reward'], label=labels[3], color=colors[3], linewidth=0.5)
    ax.plot(range(trial_num), df_dualQ[i]['model_total_reward'], label=labels[4], color=colors[4], linewidth=0.5)
    ax.plot(range(trial_num), df_rs[i]['model_total_reward'], label=labels[5], color=colors[5], linewidth=0.5)


    ax.set_title(f'participant {i+1}' , fontsize=14, fontweight='bold')
    ax.set_xlabel('trials')
    ax.set_ylabel('total reward')
    # ax.legend(frameon=False)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

for j in range(num_subplots, len(axes)):
    fig.delaxes(axes[j])
    
fig.suptitle('total rewards', fontsize=22, fontweight='bold', y=1)


# Add colored legend text under the title
x_start = 0.1
x_spacing = 0.14
y_pos = 0.975  # Slightly below suptitle

for idx, (label, color) in enumerate(zip(labels, colors)):
    fig.text(x_start + idx * x_spacing, y_pos, label, fontsize=14, color=color, fontweight='bold')




plt.tight_layout(rect=[0, 0, 1, 1]) 

# output_dir = pathlib.Path(output_dir).resolve()
# pdf_path = output_dir / "total_reward.pdf"
# svg_path = output_dir / "total_reward.svg"

# fig.savefig(pdf_path)
# fig.savefig(svg_path, format="svg", dpi=1200, bbox_inches="tight", transparent=True)

# # plt.close()


# total reward score

In [13]:
# List of model DataFrames
model_dfs = [df_wsls, df_greedy, df_softmax, df_dualQ, df_rs]
model_names = labels[1:]  # Exclude 'participants'

num_participants = len(df_participants)
num_models = len(model_dfs)

# Store scores: rows=models, cols=participants
score_matrix = np.zeros((num_models, num_participants))

for model_idx, df_model in enumerate(model_dfs):
    for p_idx in range(num_participants):
        # Get trajectories
        y_true = (df_participants[p_idx]['totalReward'] - df_participants[p_idx]['totalReward'].iloc[0]) + 10
        y_pred = df_model[p_idx]['model_total_reward']

        # Ensure lengths match
        if len(y_true) != len(y_pred):
            raise ValueError(f"Length mismatch at model {model_names[model_idx]} and participant {p_idx+1}")

        # Compute correlation (use nan_to_num in case of nan), and RMSE
        corr, _ = pearsonr(y_true, y_pred)
        corr = np.nan_to_num(corr)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))

        # Compute score
        score = abs(corr) / rmse if rmse != 0 else 0
        score_matrix[model_idx, p_idx] = score

# Plot heatmap
fig, ax = plt.subplots(figsize=(14, 3))
sns.heatmap(score_matrix, cmap='viridis', annot=False, cbar=True,
            xticklabels=[f'p{i+1}' for i in range(num_participants)],
            yticklabels=model_names, ax=ax)

ax.set_title('total reward best fit model', fontsize=16, fontweight='bold')
ax.set_xlabel('participants', fontsize=14)
ax.set_ylabel('models', fontsize=14)

plt.tight_layout()

# Save figure
# heatmap_pdf = output_dir / "total_reward_model_fit_score_heatmap.pdf"
# heatmap_svg = output_dir / "total_reward_model_fit_score_heatmap.svg"
# fig.savefig(heatmap_pdf)
# fig.savefig(heatmap_svg, format="svg", dpi=1200, bbox_inches="tight", transparent=True)



In [14]:
bar_colors = ['#E69F00', '#56B4E9', '#009E73', "#CEC10F", "#065B8D"]

filtered_indices = [i for i, name in enumerate(model_names) if name != 'wsls']
model_names_filtered = [model_names[i] for i in filtered_indices]
score_matrix_filtered = score_matrix[filtered_indices, :]  # assumes shape (models, participants)
bar_colors_filtered = [bar_colors[i] for i in filtered_indices]

# -----------------------------------------------------
# Sort models by descending mean score
model_means = {name: score_matrix_filtered[i].mean() for i, name in enumerate(model_names_filtered)}
sorted_models = sorted(model_means, key=model_means.get, reverse=True)
sorted_indices = [model_names_filtered.index(name) for name in sorted_models]

model_names_filtered = [model_names_filtered[i] for i in sorted_indices]
score_matrix_filtered = score_matrix_filtered[sorted_indices, :]
bar_colors_filtered = [bar_colors_filtered[i] for i in sorted_indices]

# -----------------------------------------------------
# Step 1: Convert filtered and sorted score_matrix to long-form DataFrame
score_data = []

for model_idx, model_name in enumerate(model_names_filtered):
    for p_idx in range(num_participants):
        score = score_matrix_filtered[model_idx, p_idx]
        score_data.append({'Model': model_name, 'Score': score})

score_df = pd.DataFrame(score_data)

# Step 2: Tukey HSD test for significance
tukey = pairwise_tukeyhsd(endog=score_df['Score'], groups=score_df['Model'], alpha=0.05)
tukey_df = pd.DataFrame(data=tukey.summary().data[1:], columns=tukey.summary().data[0])
significant_pairs = tukey_df[tukey_df['reject'] == True]

# Step 3: Plot
fig, ax = plt.subplots(figsize=(4, 4))

# Draw boxplot
sns.boxplot(data=score_df, x='Model', y='Score', color='none',
            width=0.4,
            boxprops=dict(facecolor='none', edgecolor='black', linewidth=0.666),
            whiskerprops=dict(color='black', linewidth=0.666),
            capprops=dict(color='black', linewidth=0.666),
            medianprops=dict(color='black', linewidth=0.666),
            fliersize=0,
            ax=ax)

# Scatter points per model
for i, model_name in enumerate(model_names_filtered):
    model_scores = score_df[score_df['Model'] == model_name]['Score']
    x_vals = np.random.normal(loc=i, scale=0.08, size=len(model_scores))  # jitter
    ax.scatter(x_vals, model_scores, color=bar_colors_filtered[i], s=10, alpha=0.8, edgecolors='none')

# Step 4: Significance bars
y_max = score_df['Score'].max()
y_min = score_df['Score'].min()
h = (y_max - y_min) * 0.05
bar_spacing = h * 1.5
current_y = y_max + h

for _, row in significant_pairs.iterrows():
    group1 = row['group1']
    group2 = row['group2']
    x1 = model_names_filtered.index(group1)
    x2 = model_names_filtered.index(group2)
    x_low = min(x1, x2)
    x_high = max(x1, x2)

    ax.plot([x_low, x_low, x_high, x_high],
            [current_y, current_y + h, current_y + h, current_y],
            lw=0.666, c='black')
    ax.text((x1 + x2) / 2, current_y + h / 2, "*", ha='center', va='bottom', fontsize=12, fontweight='bold')
    current_y += bar_spacing

# Final plot settings
ax.set_title("total reward similarity", fontsize=15, fontweight='bold')
ax.set_ylabel("score", fontsize=13)
ax.set_xlabel(" ", fontsize=13)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='x', length=3, width=0.666)
ax.tick_params(axis='y', width=0.666)

plt.tight_layout()

# # Save figure
# boxplot_pdf = output_dir / "total_reward_model_fit_score_boxplot.pdf"
# boxplot_svg = output_dir / "total_reward_model_fit_score_boxplot.svg"
# fig.savefig(boxplot_pdf)
# fig.savefig(boxplot_svg, format="svg", dpi=1200, bbox_inches="tight", transparent=True)
# plt.show()


In [15]:
bar_colors = ['#E69F00', '#56B4E9', '#009E73', "#CEC10F", "#065B8D"]

# Step 1: Assign each participant to the model with the highest score
best_model_indices = np.argmax(score_matrix, axis=0)
assigned_counts = np.bincount(best_model_indices, minlength=len(model_names))

# Step 2: Sort by count (descending)
sorted_indices = np.argsort(-assigned_counts)  # negative for descending

sorted_model_names = [model_names[i] for i in sorted_indices]
sorted_counts = assigned_counts[sorted_indices]
sorted_colors = [bar_colors[i] for i in sorted_indices]

# Step 3: Create sorted histogram
fig, ax = plt.subplots(figsize=(3, 3))
bars = ax.bar(sorted_model_names, sorted_counts, color=sorted_colors)

# Annotate counts
for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom', fontsize=6)

ax.set_ylabel("number of participants", fontsize=8)
ax.set_title("total reward similarity", fontsize=10, fontweight='bold')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.xticks(rotation=20)
plt.tight_layout()

# Save
# fig.savefig(output_dir / "total_reward_model_fit_score_hist.pdf")
# fig.savefig(output_dir / "total_reward_model_fit_score_hist.svg", format="svg", dpi=1200, bbox_inches="tight", transparent=True)

In [16]:

trial_num = len(df_participants[0])

# Convert all reward trajectories to NumPy arrays (as float)
participant_rewards = np.stack([
    ((df['totalReward'] - df['totalReward'].iloc[0]) + 10).astype(float).values for df in df_participants
])
wsls_rewards = np.stack([df['model_total_reward'].astype(float).values for df in df_wsls])
greedy_rewards = np.stack([df['model_total_reward'].astype(float).values for df in df_greedy])
softmax_rewards = np.stack([df['model_total_reward'].astype(float).values for df in df_softmax])
dualQ_rewards = np.stack([df['model_total_reward'].astype(float).values for df in df_dualQ])
rs_rewards = np.stack([df['model_total_reward'].astype(float).values for df in df_rs])

# Compute mean and SEM manually
def mean_and_sem(data):
    mean = np.mean(data, axis=0)
    sem_val = np.std(data, axis=0, ddof=1) / np.sqrt(data.shape[0])
    return mean, sem_val

data_groups = [participant_rewards, wsls_rewards, greedy_rewards, softmax_rewards, dualQ_rewards, rs_rewards]
labels = ['participants', 'wsls', 'greedy', 'softmax', 'dualQ', 'risk-sensitive']
colors = ['black', '#E69F00', '#56B4E9', '#009E73', "#CEC10F", "#065B8D"]

# Plot
fig, ax = plt.subplots(figsize=(5, 5))

for data, label, color in zip(data_groups, labels, colors):
    mean_vals, sem_vals = mean_and_sem(data)
    ax.plot(range(trial_num), mean_vals, label=label, color=color, linewidth=2.5)
    ax.fill_between(range(trial_num), mean_vals - sem_vals, mean_vals + sem_vals, color=color, alpha=0.3)

# Style
ax.set_title("mean total reward across participants", fontsize=12, fontweight='bold')
ax.set_xlabel("trials", fontsize=14)
ax.set_ylabel("total reward", fontsize=14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


# Custom legend (stacked vertically inside top-left corner)
x_pos = 0.2   # left padding
y_start = 0.9 # top position
y_spacing = 0.045  # vertical spacing between labels

for idx, (label, color) in enumerate(zip(labels, colors)):
    fig.text(x_pos, y_start - idx * y_spacing, label,
             fontsize=10, color=color, fontweight='bold',
             ha='left', va='top')


plt.tight_layout(rect=[0, 0, 1, 1])

# # Save
# output_dir = pathlib.Path(output_dir).resolve()
# pdf_path = output_dir / "total_reward_across_all.pdf"
# svg_path = output_dir / "total_reward_across_all.svg"

# fig.savefig(pdf_path)
# fig.savefig(svg_path, format="svg", dpi=1200, bbox_inches="tight", transparent=True)

# accuracy

In [17]:

def compute_win_rates_by_block_type(df_list, x_labels=['uniform', 'low', 'high']):
    win_rates_mix = []
    win_rates_non_mix = []

    for df in df_list:
        df['is_win'] = df['outcome'].apply(lambda x: 1 if x == 'win' else 0)

        df_mix = df[df['block_type'] == 'mix']
        df_non_mix = df[df['block_type'] != 'mix']

        if not df_mix.empty:
            total_trials_mix = df_mix.groupby('distribution').size().reset_index(name='total_trials')
            wins_mix = df_mix.groupby('distribution')['is_win'].sum().reset_index(name='wins')
            win_rate_mix = pd.merge(total_trials_mix, wins_mix, on='distribution')
            win_rate_mix['win_rate'] = win_rate_mix['wins'] / win_rate_mix['total_trials']
            win_rates_mix.append(win_rate_mix[['distribution', 'win_rate']])

        if not df_non_mix.empty:
            total_trials_non_mix = df_non_mix.groupby('distribution').size().reset_index(name='total_trials')
            wins_non_mix = df_non_mix.groupby('distribution')['is_win'].sum().reset_index(name='wins')
            win_rate_non_mix = pd.merge(total_trials_non_mix, wins_non_mix, on='distribution')
            win_rate_non_mix['win_rate'] = win_rate_non_mix['wins'] / win_rate_non_mix['total_trials']
            win_rates_non_mix.append(win_rate_non_mix[['distribution', 'win_rate']])

    def compute_group_stats(win_rates_list):
        combined = pd.concat(win_rates_list)
        mean = combined.groupby('distribution')['win_rate'].mean().reset_index()
        std = combined.groupby('distribution')['win_rate'].std().reset_index()
        stats = mean.merge(std, on='distribution', suffixes=('_mean', '_std'))
        stats['distribution'] = pd.Categorical(stats['distribution'], categories=x_labels, ordered=True)
        return stats.sort_values(by='distribution').reset_index(drop=True)

    return compute_group_stats(win_rates_mix), compute_group_stats(win_rates_non_mix), win_rates_mix, win_rates_non_mix

# Compute win rates
mean_win_rates_df_participants_mix, mean_win_rates_df_participants_non_mix, win_rates_mix, win_rates_non_mix = compute_win_rates_by_block_type(df_participants)
mean_win_rates_df_wsls_mix, mean_win_rates_df_wsls_non_mix, win_rates_wsls_mix, win_rates_wsls_non_mix = compute_win_rates_by_block_type(df_wsls)
mean_win_rates_df_greedy_mix, mean_win_rates_df_greedy_non_mix, win_rates_greedy_mix, win_rates_greedy_non_mix = compute_win_rates_by_block_type(df_greedy)
mean_win_rates_df_softmax_mix, mean_win_rates_df_softmax_non_mix, win_rates_softmax_mix, win_rates_softmax_non_mix = compute_win_rates_by_block_type(df_softmax)
mean_win_rates_df_dualQ_mix, mean_win_rates_df_dualQ_non_mix, win_rates_dualQ_mix, win_rates_dualQ_non_mix = compute_win_rates_by_block_type(df_dualQ)
mean_win_rates_df_rs_mix, mean_win_rates_df_rs_non_mix, win_rates_rs_mix, win_rates_rs_non_mix = compute_win_rates_by_block_type(df_rs)

def compute_sem(win_rates_list):
    combined = pd.concat(win_rates_list)
    sem = combined.groupby('distribution')['win_rate'].sem().reset_index()
    return sem

# Organize data
mean_dfs_fix = [mean_win_rates_df_participants_non_mix, mean_win_rates_df_wsls_non_mix, mean_win_rates_df_greedy_non_mix, mean_win_rates_df_softmax_non_mix, mean_win_rates_df_dualQ_non_mix, mean_win_rates_df_rs_non_mix]
mean_dfs_mix = [mean_win_rates_df_participants_mix, mean_win_rates_df_wsls_mix, mean_win_rates_df_greedy_mix, mean_win_rates_df_softmax_mix, mean_win_rates_df_dualQ_mix, mean_win_rates_df_rs_mix]

sem_dfs_fix = [compute_sem(lst) for lst in [win_rates_non_mix, win_rates_wsls_non_mix, win_rates_greedy_non_mix, win_rates_softmax_non_mix, win_rates_dualQ_non_mix, win_rates_rs_non_mix]]
sem_dfs_mix = [compute_sem(lst) for lst in [win_rates_mix, win_rates_wsls_mix, win_rates_greedy_mix, win_rates_softmax_mix, win_rates_dualQ_mix, win_rates_rs_mix]]

labels = ['participants', 'wsls', 'greedy', 'softmax', 'dualQ', 'risk-sensitive']
colors = ['black', '#E69F00', '#56B4E9', '#009E73', "#CEC10F", "#065B8D"]

# Plotting
x_labels = ['uniform', 'low', 'high']
bar_width = 0.12
x = np.arange(len(x_labels))

fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)
bars = []

for ax_idx, (ax, mean_dfs, sem_dfs, title) in enumerate(zip(
    axes,
    [mean_dfs_fix, mean_dfs_mix],
    [sem_dfs_fix, sem_dfs_mix],
    ['fix', 'mix']
)):

    for j, dist in enumerate(x_labels):  # for each distribution
        # Get participants bar first
        mean_p = mean_dfs[0][mean_dfs[0]['distribution'] == dist]['win_rate_mean'].values[0]
        sem_p = sem_dfs[0][sem_dfs[0]['distribution'] == dist]['win_rate'].values[0]
        bar = ax.bar(j, mean_p, width=bar_width, color=colors[0], yerr=sem_p, capsize=5, label=labels[0] if ax_idx == 0 and j == 0 else None)
        if ax_idx == 0 and j == 0:
            bars.append(bar[0])

        # Gather and sort all models except participants
        dist_vals = []
        for i in range(1, len(mean_dfs)):
            mean_val = mean_dfs[i][mean_dfs[i]['distribution'] == dist]['win_rate_mean'].values[0]
            sem_val = sem_dfs[i][sem_dfs[i]['distribution'] == dist]['win_rate'].values[0]
            dist_vals.append((labels[i], colors[i], mean_val, sem_val))

        dist_vals.sort(key=lambda x: x[2], reverse=True)  # sort by win_rate

        for k, (label, color, mean_val, sem_val) in enumerate(dist_vals):
            bar = ax.bar(j + (k + 1) * bar_width, mean_val, width=bar_width, color=color,
                         yerr=sem_val, capsize=5, label=label if ax_idx == 0 and j == 0 else None)
            if ax_idx == 0 and j == 0:
                bars.append(bar[0])

    ax.set_title(f'{title}', fontweight='bold')
    ax.set_xticks(np.arange(len(x_labels)) + ((len(mean_dfs)) / 2) * bar_width / 2)
    ax.set_xticklabels(x_labels)
    ax.set_ylabel('accuracy')
    ax.set_ylim(0, 1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

fig.legend(bars, labels, loc='upper center', ncol=4, frameon=False, bbox_to_anchor=(0.5, 0.9))
plt.suptitle('accuracy by distribution', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.88])

# Save figure
# output_dir = pathlib.Path(output_dir).resolve()
# pdf_path = output_dir / "accuracy_fix_mix.pdf"
# svg_path = output_dir / "accuracy_fix_mix.svg"

# fig.savefig(pdf_path)
# fig.savefig(svg_path, format="svg", dpi=1200, bbox_inches="tight", transparent=True)


## participants accuracy

In [18]:
# --- Participants-only grouped boxplots: 3 dists × (fix, mix) ---

# Config
x_labels = ['uniform', 'low', 'high']  # group order
scatter_colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

rng = np.random.default_rng(42)  # deterministic jitter
box_width = 0.28                  # width for each box
offset = 0.18                     # half-spacing between fix/mix within a group

def _one_df_winrates_by_dist(df):
    d = df.copy()
    d['is_win'] = (d['outcome'] == 'win').astype(int)
    g = d.groupby('distribution', observed=True)['is_win'].mean()
    return g.reindex(x_labels)

def _compute_winrates_by_block(df_list, block_selector):
    """
    For a list of participant DFs, return a list of Series of win rates by distribution
    for a chosen block ('fix' or 'mix').
    """
    out = []
    for df in df_list:
        if not isinstance(df, pd.DataFrame) or df.empty:
            continue
        sub = df[df['block_type'] == 'mix'] if block_selector == 'mix' else df[df['block_type'] != 'mix']
        if sub.empty:
            continue
        out.append(_one_df_winrates_by_dist(sub))
    return out  # list of Series indexed by distribution

# Collect per-participant win rates for participants only
wr_fix_series = _compute_winrates_by_block(df_participants, 'fix')
wr_mix_series = _compute_winrates_by_block(df_participants, 'mix')

# For each distribution, gather the vector of participant accuracies (fix and mix)
data_fix = {dist: [] for dist in x_labels}
data_mix = {dist: [] for dist in x_labels}

for s in wr_fix_series:
    for dist in x_labels:
        v = s.get(dist)
        if pd.notna(v):
            data_fix[dist].append(float(v))

for s in wr_mix_series:
    for dist in x_labels:
        v = s.get(dist)
        if pd.notna(v):
            data_mix[dist].append(float(v))

# ---- Plot ----
fig, ax = plt.subplots(figsize=(4,4), tight_layout=True)

group_centers = np.arange(len(x_labels))  # 0,1,2
pos_fix = group_centers - offset
pos_mix = group_centers + offset

# Draw boxplots (unfilled, black edges), per distribution
for j, dist in enumerate(x_labels):
    # Fix
    y_fix = data_fix[dist]
    if len(y_fix) > 0:
        bp1 = ax.boxplot(
            y_fix,
            positions=[pos_fix[j]],
            widths=box_width,
            patch_artist=True,
            showfliers=False,
            boxprops=dict(facecolor='none', edgecolor='black', linewidth=0.7),
            medianprops=dict(color='black', linewidth=0.7),
            whiskerprops=dict(color='black', linewidth=0.7),
            capprops=dict(color='black', linewidth=0.7)
        )
        # Scatter overlay (color by distribution)
        jitter = rng.normal(loc=0.0, scale=0.02, size=len(y_fix))
        ax.scatter(
            np.full(len(y_fix), pos_fix[j]) + jitter,
            y_fix,
            s=12,
            alpha=0.7,
            color=scatter_colors[dist],
            edgecolors='none',
            zorder=3
        )

    # Mix
    y_mix = data_mix[dist]
    if len(y_mix) > 0:
        bp2 = ax.boxplot(
            y_mix,
            positions=[pos_mix[j]],
            widths=box_width,
            patch_artist=True,
            showfliers=False,
            boxprops=dict(facecolor='none', edgecolor='black', linewidth=0.7),
            medianprops=dict(color='black', linewidth=0.7),
            whiskerprops=dict(color='black', linewidth=0.7),
            capprops=dict(color='black', linewidth=0.7)
        )
        jitter = rng.normal(loc=0.0, scale=0.02, size=len(y_mix))
        ax.scatter(
            np.full(len(y_mix), pos_mix[j]) + jitter,
            y_mix,
            s=12,
            alpha=0.7,
            color=scatter_colors[dist],
            edgecolors='none',
            zorder=3
        )

# Axes/labels
# ax.set_xlim(-0.75, len(x_labels) - 0.25)
ax.set_ylim(0, 1)
ax.set_ylabel('accuracy', fontsize=10)
# ax.set_xticks(group_centers, x_labels, fontsize=10)


group_centers = np.arange(len(x_labels))  # 0,1,2
pos_fix = group_centers - offset
pos_mix = group_centers + offset

# First level: fix/mix ticks
tick_positions = np.r_[pos_fix, pos_mix]          # [uniform-fix, uniform-mix, low-fix, low-mix, high-fix, high-mix]
tick_labels    = ['fix', 'mix'] * len(x_labels)   # alternating labels

ax.set_xticks(tick_positions, tick_labels, fontsize=9)

# Second level: distribution labels centered under each pair
for j, dist in enumerate(x_labels):
    ax.text(
        group_centers[j], -0.12, dist,
        ha="center", va="top", transform=ax.get_xaxis_transform(),
        fontsize=10, fontweight="bold"
    )

# Adjust bottom margin so distribution labels fit
fig.subplots_adjust(bottom=0.18)




# Style
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
for spine in ax.spines.values():
    spine.set_linewidth(0.6)

ax.set_title('accuracy in fix vs mix', fontsize=12)

# Save
pdf_path = pathlib.Path(output_dir).resolve() / "participants_accuracy_fix_mix.pdf"
svg_path = pathlib.Path(output_dir).resolve() / "participants_accuracy_fix_mix.svg"
fig.savefig(pdf_path, bbox_inches="tight", transparent=True)
fig.savefig(svg_path, format="svg", dpi=1200, bbox_inches="tight", transparent=True)


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from scipy.stats import ttest_ind

# ------------------------------------------
# Function to compute win rates
def compute_win_rates(df_list):
    win_rates = []

    for df in df_list:
        if not isinstance(df, pd.DataFrame):
            continue
        df = df.copy()
        df['is_win'] = df['outcome'].apply(lambda x: 1 if x == 'win' else 0)
        grouped = df.groupby('distribution')['is_win'].mean().reset_index()
        grouped.columns = ['distribution', 'win_rate']
        win_rates.append(grouped)

    return win_rates

# ------------------------------------------
# Configuration
x_labels = ['uniform', 'low', 'high']
labels = ['participants','greedy', 'softmax', 'dualQ', 'risk-sensitive']
colors = ['black', '#56B4E9', '#009E73', "#CEC10F", "#065B8D"]
output_dir = pathlib.Path(output_dir).resolve()

# ------------------------------------------
# Compute win rates
win_rates_all = [
    compute_win_rates(df_participants),
    compute_win_rates(df_greedy),
    compute_win_rates(df_softmax),
    compute_win_rates(df_dualQ),
    compute_win_rates(df_rs),
]

# ------------------------------------------
# Create long-form DataFrame
plot_data = []

for label, color, model_rates in zip(labels, colors, win_rates_all):
    for dist in x_labels:
        for df in model_rates:
            sub = df[df['distribution'] == dist]
            if not sub.empty:
                win = sub['win_rate'].values[0]
                plot_data.append({'Model': label, 'Distribution': dist, 'WinRate': win, 'Color': color})

plot_df = pd.DataFrame(plot_data)
# Annotate significance above each model vs participants
comparisons = ['greedy', 'softmax', 'dualQ', 'risk-sensitive']
# ------------------------------------------
# ------------------------------------------
# Plot boxplots: 1 subplot per distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 3.5), sharey=True)
bar_width = 0.8

for i, dist in enumerate(x_labels):
    ax = axes[i]
    dist_df = plot_df[plot_df['Distribution'] == dist]
    x_positions = np.arange(len(labels))

    for j, label in enumerate(labels):
        model_df = dist_df[dist_df['Model'] == label]
        y_vals = model_df['WinRate'].values

        # Boxplot
        bp = ax.boxplot(
            y_vals, positions=[x_positions[j]], widths=bar_width * 0.8,
            patch_artist=True, showfliers=False,
            boxprops=dict(facecolor='none', edgecolor='black', linewidth=0.5),
            medianprops=dict(color='black', linewidth=0.5),
            whiskerprops=dict(color='black', linewidth=0.5),
            capprops=dict(color='black', linewidth=0.5)
        )

        # Scatter overlay with jitter
        jitter = np.random.normal(loc=0, scale=0.1, size=len(y_vals))
        ax.scatter(x_positions[j] + jitter, y_vals, color=colors[j], s=15,
                   zorder=3, alpha=0.4, edgecolors='none')

    # Dashed vertical line between participants (index 0) and others
    ax.axvline(x=0.5, color='gray', linestyle='--', linewidth=0.5)

    # X-axis formatting
    ax.set_xticks(x_positions)
    ax.set_xticklabels(labels, rotation=45, ha='right', fontsize=8)
    ax.set_title(f'{dist}', fontweight='bold')

    # Significance annotations
    p_vals = dist_df[dist_df['Model'] == 'participants']['WinRate'].values
    for model in comparisons:
        m_vals = dist_df[dist_df['Model'] == model]['WinRate'].values
        t_stat, p_val = ttest_ind(p_vals, m_vals, equal_var=False)
        signif_text = "*" if p_val < (0.05 / 5) else "n.s."
        color = 'black' if signif_text == "*" else 'black'

        j = labels.index(model)
        y = dist_df[dist_df['Model'] == model]['WinRate'].max()
        ax.text(x_positions[j], 0.91, signif_text, ha='center', va='bottom',
                fontsize=8, color=color, fontweight='bold')

    # Styling
    ax.set_ylim(0, 1)
    if i == 0:
        ax.set_ylabel("accuracy")
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    for spine in ax.spines.values():
        spine.set_linewidth(0.5)

# Legend
handles = [plt.Line2D([0], [0], marker='o', linestyle='', color=color, label=label)
           for label, color in zip(labels, colors)]
fig.legend(handles=handles, loc='upper center', ncol=5, frameon=False, bbox_to_anchor=(0.5, 1.05))

plt.tight_layout(rect=[0, 0, 1, 1.02])

# Save
# pdf_path = output_dir / "accuracy.pdf"
# svg_path = output_dir / "accuracy.svg"
# fig.savefig(pdf_path, bbox_inches="tight", transparent=True)
# fig.savefig(svg_path, format="svg", dpi=1200, bbox_inches="tight", transparent=True)
# plt.show()


# arrow up percentage fix and mix combined

In [20]:
def sigmoid3(x, L, x0, k):
    return L / (1 + np.exp(-k * (x - x0)))

# -----------------------------------------------
# Distributions and colors
distributions = ['uniform', 'low', 'high']
colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

# Models: list of (list of subject-DataFrames, label, column name for choice)
models = [
    (df_participants, 'participants', 'choice'),
    (df_greedy,       'greedy',       'model_choices'),
    (df_softmax,      'softmax',      'model_choices'),
    (df_dualQ,        'dualQ',        'model_choices'),
    (df_rs,           'risk-sensitive','model_choices'),
]

# -----------------------------------------------
# Compute arrow-up percentages
def compute_arrowup_stats(subject_dfs, dist, choice_col):
    all_subject_percentages = []

    for df in subject_dfs:
        sub_df = df[df['distribution'] == dist].copy()
        if sub_df.empty: continue

        if choice_col != 'choice':
            sub_df[choice_col] = sub_df[choice_col].map({1: 'arrowup', 0: 'arrowdown'})

        sub_df['group'] = sub_df['myCard'].astype(str)
        group_counts = sub_df.groupby(['group', choice_col], observed=True)[choice_col].count().unstack(fill_value=0)
        group_totals = group_counts.sum(axis=1)
        group_percentages = (group_counts.T / group_totals).T
        group_percentages['group'] = group_percentages.index
        all_subject_percentages.append(group_percentages.reset_index(drop=True))

    if not all_subject_percentages:
        return None, None, None

    combined_df = pd.concat(all_subject_percentages).fillna(0)
    mean_df = combined_df.groupby('group', observed=True).mean().reset_index()
    sem_df  = combined_df.groupby('group', observed=True).sem().reset_index()
    return mean_df, sem_df, mean_df['group']

# -----------------------------------------------
# Storage for sigmoid parameters
fit_params = defaultdict(lambda: defaultdict(dict))
param_list = []

# -----------------------------------------------
# Plotting setup
fig, axes = plt.subplots(1, 5, figsize=(15, 3.5), dpi=130, sharey=True)

for col, (subject_dfs, label, choice_col) in enumerate(models):
    for dist in distributions:
        mean_df, _, groups = compute_arrowup_stats(subject_dfs, dist, choice_col)
        if mean_df is None:
            continue

        arrow_up = mean_df.get('arrowup', 0).values
        x = np.arange(len(groups))

        if len(np.unique(arrow_up)) > 2:
            try:
                p0 = [max(arrow_up), np.median(x), 1]
                popt, _ = curve_fit(sigmoid3, x, arrow_up, p0, maxfev=800000)
                fit_params[dist][label] = popt
                param_list.append((dist, label, popt))
            except (RuntimeError, OptimizeWarning, ValueError) as e:
                print(f"Fit failed for {label} | dist: {dist}: {e}")

# -----------------------------------------------
# Standardize parameters
all_vecs = [params for (_, _, params) in param_list]
scaler = StandardScaler()
scaled_vecs = scaler.fit_transform(all_vecs)

param_lookup = {}
for (dist, label, _), scaled_vec in zip(param_list, scaled_vecs):
    param_lookup[(dist, label)] = scaled_vec

# -----------------------------------------------
# Plot each model's arrow-up curve
for col, (subject_dfs, label, choice_col) in enumerate(models):
    ax = axes[col]
    for dist in distributions:
        mean_df, sem_df, groups = compute_arrowup_stats(subject_dfs, dist, choice_col)
        if mean_df is None:
            continue

        arrow_up = mean_df.get('arrowup', 0).values
        sem_up   = sem_df.get('arrowup', 0).values
        x = np.arange(len(groups))
        color = colors[dist]

        ax.errorbar(x, arrow_up, yerr=sem_up, fmt='o', color=color,
                    capsize=1, capthick=0.2, markersize=1.5, elinewidth=0.3, label=dist)

        popt = fit_params[dist].get(label)
        if popt is not None:
            x_smooth = np.linspace(x.min(), x.max(), 200)
            y_smooth = sigmoid3(x_smooth, *popt)
            ax.plot(x_smooth, y_smooth, color=color, linewidth=1, alpha=0.6, linestyle='--')

    ax.set_xticks(x)
    ax.set_xticklabels(groups)

    ax.set_ylim(0, 1)
    ax.set_yticks([i/100 for i in range(0, 101, 20)])  # actual positions: 0.0 to 1.0
    ax.set_yticklabels([f"{i}" for i in range(0, 101, 20)])  # display as 0%, 20%, ..., 100%
    
    ax.set_xlabel("myCard", fontsize=10)
    ax.set_ylabel("arrow‑up (%)", fontsize=12, fontweight='bold')
    ax.set_title(label, fontsize=9.5, weight='bold')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# -----------------------------------------------
# Legend
handles = [Patch(facecolor="none", edgecolor="none", label=f"{d}") for d in distributions]
legend = fig.legend(handles=handles, loc='lower center', bbox_to_anchor=(0.5, 0.86),
                    ncol=3, frameon=False, handlelength=0)
for text, dist in zip(legend.get_texts(), distributions):
    text.set_color(colors[dist])

plt.tight_layout(rect=[0, 0, 1, 0.92])
plt.suptitle('arrow up percentage', fontsize=16, fontweight='bold', y=1)

# -----------------------------------------------
# Save
# output_dir = pathlib.Path(output_dir).resolve()
# fig.savefig(output_dir / "arrowup_percentage.pdf")
# fig.savefig(output_dir / "arrowup_percentage.svg", format="svg", dpi=1200, bbox_inches="tight", transparent=True)


Text(0.5, 1, 'arrow up percentage')

# fix and mix separately

In [21]:

# -----------------------------------------------
# Sigmoid
def sigmoid3(x, L, x0, k):
    return L / (1 + np.exp(-k * (x - x0)))

# -----------------------------------------------
# Distributions and colors
distributions = ['uniform', 'low', 'high']
colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

# -----------------------------------------------
# Models: ONLY RS, participants, greedy, softmax
models = [
    (df_participants, 'participants',    'choice'),
        (df_rs,           'risk-sensitive', 'model_choices'),
    (df_greedy,       'greedy',          'model_choices'),
    (df_softmax,      'softmax',         'model_choices'),
]

# -----------------------------------------------
# Compute arrow-up percentages for one block + one distribution
def compute_arrowup_stats(subject_dfs, dist, choice_col, block_label):
    """
    Returns (mean_df, sem_df, groups) for arrow-up proportions per myCard, within a single distribution,
    restricted to rows where block_type == block_label ('fix' or 'mix').
    """
    all_subject_percentages = []

    for df in subject_dfs:
        d = df.copy()
        d = d[d['block_type'] == block_label]  # filter by block
        if d.empty:
            continue

        sub_df = d[d['distribution'] == dist].copy()
        if sub_df.empty:
            continue

        if choice_col != 'choice':
            sub_df[choice_col] = sub_df[choice_col].map({1: 'arrowup', 0: 'arrowdown'})

        sub_df['group'] = sub_df['myCard'].astype(str)
        gc = sub_df.groupby(['group', choice_col], observed=True)[choice_col].count().unstack(fill_value=0)
        gt = gc.sum(axis=1)
        gp = (gc.T / gt).T
        gp['group'] = gp.index
        all_subject_percentages.append(gp.reset_index(drop=True))

    if not all_subject_percentages:
        return None, None, None

    combined_df = pd.concat(all_subject_percentages).fillna(0)
    mean_df = combined_df.groupby('group', observed=True).mean().reset_index()
    sem_df  = combined_df.groupby('group', observed=True).sem().reset_index()
    return mean_df, sem_df, mean_df['group']

# -----------------------------------------------
# Fit sigmoids for a block; return standardized param lookup
def fit_params_for_block(block_label):
    fit_params = defaultdict(lambda: defaultdict(dict))   # fit_params[dist][label] -> (L, x0, k)
    param_list = []

    for (subject_dfs, label, choice_col) in models:
        for dist in distributions:
            mean_df, _, groups = compute_arrowup_stats(subject_dfs, dist, choice_col, block_label)
            if mean_df is None:
                continue
            arrow_up = mean_df.get('arrowup', 0).values
            x = np.arange(len(groups))
            if len(np.unique(arrow_up)) > 2:
                try:
                    p0 = [max(arrow_up), np.median(x), 1.0]
                    popt, _ = curve_fit(sigmoid3, x, arrow_up, p0=p0, maxfev=800000)
                    fit_params[dist][label] = popt
                    param_list.append((dist, label, popt))
                except (RuntimeError, OptimizeWarning, ValueError) as e:
                    print(f"[{block_label}] Fit failed for {label} | dist: {dist}: {e}")

    # Standardize (L, x0, k) across all (dist, model) in this block
    param_lookup = {}
    if param_list:
        scaler = StandardScaler()
        all_vecs = [p for (_, _, p) in param_list]
        scaled = scaler.fit_transform(all_vecs)
        for (key, vec) in zip([(d, l) for (d, l, _) in param_list], scaled):
            param_lookup[key] = vec

    return fit_params, param_lookup

# -----------------------------------------------
# Prepare figure: 2 rows (fix, mix) × 4 columns (models)
n_models = len(models)
fig, axes = plt.subplots(2, n_models, figsize=(3.4*n_models, 7.2), dpi=130, sharey=True)

# Fit once per block (for smoother curves)
fit_fix,  param_lookup_fix  = fit_params_for_block('fix')
fit_mix,  param_lookup_mix  = fit_params_for_block('mix')

# Plotting helper per cell
def plot_cell(ax, subject_dfs, label, choice_col, block_label, fits):
    groups_for_ticks = None
    for dist in distributions:
        mean_df, sem_df, groups = compute_arrowup_stats(subject_dfs, dist, choice_col, block_label)
        if mean_df is None:
            continue

        arrow_up = mean_df.get('arrowup', 0).values
        sem_up   = sem_df.get('arrowup', 0).values
        x = np.arange(len(groups))
        groups_for_ticks = groups

        ax.errorbar(
            x, arrow_up, yerr=sem_up, fmt='o', color=colors[dist],
            capsize=1, capthick=0.2, markersize=1.5, elinewidth=0.3, label=dist
        )

        popt = fits[dist].get(label)
        if popt is not None:
            x_smooth = np.linspace(x.min(), x.max(), 200)
            y_smooth = sigmoid3(x_smooth, *popt)
            ax.plot(x_smooth, y_smooth, color=colors[dist], linewidth=1, alpha=0.6, linestyle='--')

    if groups_for_ticks is not None:
        x = np.arange(len(groups_for_ticks))
        ax.set_xticks(x)
        ax.set_xticklabels(groups_for_ticks)

    ax.set_ylim(0, 1)
    ax.set_yticks([i/100 for i in range(0, 101, 20)])
    ax.set_yticklabels([f"{i}" for i in range(0, 101, 20)])

# Row 0 = FIX, Row 1 = MIX
for col, (subject_dfs, label, choice_col) in enumerate(models):
    # FIX row
    ax0 = axes[0, col]
    plot_cell(ax0, subject_dfs, label, choice_col, 'fix', fit_fix)
    ax0.set_title(f"{label}", fontsize=10, weight='bold')
    ax0.spines['top'].set_visible(False); ax0.spines['right'].set_visible(False)
    if col == 0:
        ax0.set_ylabel("arrow-up (%)\n[fix]", fontsize=11, fontweight='bold')
    ax0.set_xlabel("myCard", fontsize=10)

    # MIX row
    ax1 = axes[1, col]
    plot_cell(ax1, subject_dfs, label, choice_col, 'mix', fit_mix)
    ax1.spines['top'].set_visible(False); ax1.spines['right'].set_visible(False)
    if col == 0:
        ax1.set_ylabel("arrow-up (%)\n[mix]", fontsize=11, fontweight='bold')
    ax1.set_xlabel("myCard", fontsize=10)

# Legend (one for the whole figure)
handles = [Patch(facecolor="none", edgecolor="none", label=f"{d}") for d in distributions]
legend = fig.legend(handles=handles, loc='upper center', bbox_to_anchor=(0.5, 1.02),
                    ncol=3, frameon=False, handlelength=0)
for text, dist in zip(legend.get_texts(), distributions):
    text.set_color(colors[dist])

plt.suptitle('arrow up percentage in fix (top), mix (bottom)', fontsize=14, fontweight='bold', y=1.06)
plt.tight_layout(rect=[0, 0, 1, 0.98])

# -----------------------------------------------
# SAVE
try:
    output_dir  # if already defined by you elsewhere
except NameError:
    output_dir = Path(".")
else:
    output_dir = Path(output_dir)

output_dir.mkdir(parents=True, exist_ok=True)
fig.savefig(output_dir / "arrowup_percentage_fix_mix.pdf", bbox_inches="tight", transparent=True)
fig.savefig(output_dir / "arrowup_percentage_fix_mix.svg", format="svg", dpi=1200, bbox_inches="tight", transparent=True)



## participants arrow up percentages for fix and mix

In [22]:
def sigmoid3(x, L, x0, k):
    return L / (1 + np.exp(-k * (x - x0)))

# -----------------------------------------------
# Distributions and colors
distributions = ['uniform', 'low', 'high']
colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

# -----------------------------------------------
# Only PARTICIPANTS
models = [
    (df_participants, 'participants', 'choice'),
]

# -----------------------------------------------
# Compute arrow-up percentages for one block + one distribution
def compute_arrowup_stats(subject_dfs, dist, choice_col, block_label):
    """
    Returns (mean_df, sem_df, groups) for arrow-up proportions per myCard, within a single distribution,
    restricted to rows where block_type == block_label ('fix' or 'mix').
    """
    all_subject_percentages = []

    for df in subject_dfs:
        d = df.copy()
        d = d[d['block_type'] == block_label]  # filter by block
        if d.empty:
            continue

        sub_df = d[d['distribution'] == dist].copy()
        if sub_df.empty:
            continue

        # participants use 'choice' with string values
        if choice_col != 'choice':
            sub_df[choice_col] = sub_df[choice_col].map({1: 'arrowup', 0: 'arrowdown'})

        sub_df['group'] = sub_df['myCard'].astype(str)
        gc = sub_df.groupby(['group', choice_col], observed=True)[choice_col].count().unstack(fill_value=0)
        gt = gc.sum(axis=1)
        gp = (gc.T / gt).T
        gp['group'] = gp.index
        all_subject_percentages.append(gp.reset_index(drop=True))

    if not all_subject_percentages:
        return None, None, None

    combined_df = pd.concat(all_subject_percentages).fillna(0)
    mean_df = combined_df.groupby('group', observed=True).mean().reset_index()
    sem_df  = combined_df.groupby('group', observed=True).sem().reset_index()
    return mean_df, sem_df, mean_df['group']

# -----------------------------------------------
# Fit sigmoids for a block; return standardized param lookup (kept for consistency)
def fit_params_for_block(block_label):
    fit_params = defaultdict(lambda: defaultdict(dict))   # fit_params[dist][label] -> (L, x0, k)
    param_list = []

    for (subject_dfs, label, choice_col) in models:
        for dist in distributions:
            mean_df, _, groups = compute_arrowup_stats(subject_dfs, dist, choice_col, block_label)
            if mean_df is None:
                continue
            arrow_up = mean_df.get('arrowup', 0).values
            x = np.arange(len(groups))
            if len(np.unique(arrow_up)) > 2:
                try:
                    p0 = [max(arrow_up), np.median(x), 1.0]
                    popt, _ = curve_fit(sigmoid3, x, arrow_up, p0=p0, maxfev=800000)
                    fit_params[dist][label] = popt
                    param_list.append((dist, label, popt))
                except (RuntimeError, OptimizeWarning, ValueError) as e:
                    print(f"[{block_label}] Fit failed for {label} | dist: {dist}: {e}")

    # Standardize (L, x0, k) across all (dist, model) in this block
    param_lookup = {}
    if param_list:
        scaler = StandardScaler()
        all_vecs = [p for (_, _, p) in param_list]
        scaled = scaler.fit_transform(all_vecs)
        for (key, vec) in zip([(d, l) for (d, l, _) in param_list], scaled):
            param_lookup[key] = vec

    return fit_params, param_lookup

# -----------------------------------------------
# Prepare figure: 1 row × 2 columns (fix, mix), PARTICIPANTS ONLY
fig, axes = plt.subplots(1, 2, figsize=(5,3), dpi=130, sharey=True)

# Fit once per block (for smoother curves)
fit_fix, param_lookup_fix = fit_params_for_block('fix')
fit_mix, param_lookup_mix = fit_params_for_block('mix')

# Plotting helper per block (participants only)
def plot_block(ax, subject_dfs, choice_col, block_label, fits):
    groups_for_ticks = None
    label = 'participants'  # fixed
    for dist in distributions:
        mean_df, sem_df, groups = compute_arrowup_stats(subject_dfs, dist, choice_col, block_label)
        if mean_df is None:
            continue

        arrow_up = mean_df.get('arrowup', 0).values
        sem_up   = sem_df.get('arrowup', 0).values if sem_df is not None else np.zeros_like(arrow_up)
        x = np.arange(len(groups))
        groups_for_ticks = groups

        ax.errorbar(
            x, arrow_up, yerr=sem_up, fmt='o', color=colors[dist],
            capsize=2, capthick=0.2, markersize=2.0, elinewidth=0.4, label=dist
        )

        popt = fits[dist].get(label)
        if popt is not None:
            x_smooth = np.linspace(x.min(), x.max(), 200)
            y_smooth = sigmoid3(x_smooth, *popt)
            ax.plot(x_smooth, y_smooth, color=colors[dist], linewidth=1, alpha=0.7, linestyle='--')

    if groups_for_ticks is not None:
        x = np.arange(len(groups_for_ticks))
        ax.set_xticks(x)
        ax.set_xticklabels(groups_for_ticks)

    ax.set_ylim(0, 1)
    ax.set_yticks([i/100 for i in range(0, 101, 20)])
    ax.set_yticklabels([f"{i}" for i in range(0, 101, 20)])
    ax.set_xlabel("myCard", fontsize=10)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# Left = FIX, Right = MIX
(subject_dfs, label, choice_col) = models[0]

ax_fix = axes[0]
plot_block(ax_fix, subject_dfs, choice_col, 'fix', fit_fix)
ax_fix.set_title("fix", fontsize=8)
ax_fix.set_ylabel("arrow-up (%)", fontsize=11, fontweight='bold')

ax_mix = axes[1]
plot_block(ax_mix, subject_dfs, choice_col, 'mix', fit_mix)
ax_mix.set_title("mix", fontsize=8)

# Legend (one for the whole figure)
handles = [Patch(facecolor="none", edgecolor="none", label=f"{d}") for d in distributions]
legend = fig.legend(handles=handles, loc='upper center', bbox_to_anchor=(0.5, 1.08),
                    ncol=3, frameon=False, handlelength=0)
for text, dist in zip(legend.get_texts(), distributions):
    text.set_color(colors[dist])

plt.suptitle('arrow up percentage', fontsize=13, fontweight='bold')
plt.tight_layout(rect=[0, 0, 1, 0.98])

# -----------------------------------------------
# SAVE
try:
    output_dir  # if already defined by you elsewhere
except NameError:
    output_dir = Path(".")
else:
    output_dir = Path(output_dir)

output_dir.mkdir(parents=True, exist_ok=True)
fig.savefig(output_dir / "arrowup_percentage_fix_mix_participants.pdf", bbox_inches="tight", transparent=True)
fig.savefig(output_dir / "arrowup_percentage_fix_mix_participants.svg", format="svg", dpi=1200, bbox_inches="tight", transparent=True)


## compare curves in fix and mix arrow up percentages

In [23]:

def sigmoid3(x, L, x0, k):
    return L / (1 + np.exp(-k * (x - x0)))

# -------------------------------------------------
# Config
distributions = ['uniform', 'low', 'high']
blocks        = ['fix', 'mix']

models = [
    (df_participants, 'participant',    'choice'),
    (df_greedy,       'greedy',         'model_choices'),
    (df_softmax,      'softmax',        'model_choices'),
    (df_rs,           'risk-sensitive', 'model_choices'),
]

dist_colors  = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

# -------------------------------------------------
# Fit a single subject × distribution × block
def fit_subject_distribution_block(df, dist, choice_col, block_label):
    d = df[df['block_type'] == block_label]
    if d.empty:
        return None

    sub_df = d[d['distribution'] == dist].copy()
    if sub_df.empty:
        return None

    # normalize choice col to arrowup/arrowdown if needed
    if choice_col != 'choice':
        if sub_df[choice_col].dtype.kind in 'biufc':
            sub_df[choice_col] = sub_df[choice_col].map({1: 'arrowup', 0: 'arrowdown'})
    sub_df['group'] = sub_df['myCard'].astype(str)

    # counts per card
    gc = sub_df.groupby(['group', choice_col], observed=True)[choice_col].count().unstack(fill_value=0)
    if gc.empty:
        return None

    gt = gc.sum(axis=1).replace(0, np.nan)
    gp = (gc.T / gt).T
    if 'arrowup' not in gp.columns:
        return None

    arrow_up = gp['arrowup'].fillna(0).values
    if arrow_up.size < 3:
        return None

    x = np.arange(len(arrow_up))
    try:
        p0 = [max(arrow_up), np.median(x), 0.1]
        popt, _ = curve_fit(
            sigmoid3, x, arrow_up,
            p0=p0, maxfev=20000,
            bounds=([0, 0, -10], [1.5, 10, 10])
        )
        return popt
    except Exception:
        return None

# -------------------------------------------------
# Fit all subjects per block
def fit_all_by_block(block_label):
    fit_dfs = {}
    for subject_dfs, label, choice_col in models:
        rows = []
        for i, df in enumerate(subject_dfs):
            subj_idx = i
            if 'subject_idx' in df.columns:
                try:
                    uniq = pd.unique(df['subject_idx'].dropna())
                    if len(uniq) == 1:
                        subj_idx = int(uniq[0])
                except Exception:
                    pass
            for dist in distributions:
                popt = fit_subject_distribution_block(df, dist, choice_col, block_label)
                if popt is not None:
                    rows.append({
                        'subject_idx': subj_idx,
                        'distribution': dist,
                        'L':  popt[0],
                        'x0': popt[1],
                        'k':  popt[2],
                    })
        fit_dfs[f'sigmoid_fit_{label}'] = pd.DataFrame(
            rows, columns=['subject_idx', 'distribution', 'L', 'x0', 'k']
        ) if rows else pd.DataFrame(columns=['subject_idx', 'distribution', 'L', 'x0', 'k'])
    return fit_dfs

# -------------------------------------------------
# Run fits
fits_fix = fit_all_by_block('fix')
fits_mix = fit_all_by_block('mix')

part_fix = fits_fix['sigmoid_fit_participant']
part_mix = fits_mix['sigmoid_fit_participant']

# -------------------------------------------------
# Compute dissimilarity (RMSE)
def rmse(a, b):
    a = np.asarray(a); b = np.asarray(b)
    return np.sqrt(np.mean((a - b) ** 2))

def compute_fix_mix_dissimilarities(part_fix_df, part_mix_df, dists=('uniform','low','high')):
    cards = np.arange(1, 10)
    rows = []
    for dist in dists:
        fix_sub = part_fix_df[part_fix_df['distribution'] == dist]
        mix_sub = part_mix_df[part_mix_df['distribution'] == dist]
        subj_with_both = sorted(set(fix_sub['subject_idx']).intersection(set(mix_sub['subject_idx'])))
        for subj in subj_with_both:
            f_row = fix_sub[fix_sub['subject_idx'] == subj]
            m_row = mix_sub[mix_sub['subject_idx'] == subj]
            if f_row.empty or m_row.empty:
                continue
            f_params = f_row[['L','x0','k']].iloc[0].values
            m_params = m_row[['L','x0','k']].iloc[0].values
            f_hat = sigmoid3(cards, *f_params)
            m_hat = sigmoid3(cards, *m_params)
            rows.append({'subject_idx': subj, 'distribution': dist, 'rmse': rmse(f_hat, m_hat)})
    return pd.DataFrame(rows)

df_fixmix = compute_fix_mix_dissimilarities(part_fix, part_mix, distributions)

# -------------------------------------------------
# Permutation test (sign-flip null) for mean(RMSE) > 0
def permutation_test_mean_gt0(data, n_perm=10000, seed=42):
    rng = np.random.default_rng(seed)
    data = np.asarray(data)
    if len(data) == 0:
        return np.nan
    obs_mean = np.mean(data)
    signs = rng.choice([-1, 1], size=(n_perm, len(data)))
    perm_means = np.mean(data * signs, axis=1)
    return np.mean(perm_means >= obs_mean)  # one-sided p-value

dist_order = ['uniform', 'low', 'high']
perm_pvals = []
for dist in dist_order:
    dis_vals = df_fixmix.loc[df_fixmix['distribution'] == dist, 'rmse'].dropna().values
    if len(dis_vals) > 1:
        p_val = permutation_test_mean_gt0(dis_vals, n_perm=10000, seed=42)
    else:
        p_val = np.nan
    perm_pvals.append(p_val)

# Multiple hypothesis correction (Bonferroni)
rej, pvals_corrected, _, _ = multipletests(
    [p if not np.isnan(p) else 1.0 for p in perm_pvals],
    alpha=0.05, method='bonferroni'
)

# -------------------------------------------------
# Convert to similarity for plotting
df_fixmix['similarity'] = 1.0 - df_fixmix['rmse']

sim_data   = [df_fixmix.loc[df_fixmix['distribution'] == d, 'similarity'].dropna().values for d in dist_order]
positions  = np.arange(len(dist_order))

fig, ax = plt.subplots(figsize=(4,4))

bp = ax.boxplot(
    sim_data,
    positions=positions,
    widths=0.55,
    patch_artist=True,
    showfliers=False,
    boxprops=dict(facecolor='none', edgecolor='black', linewidth=0.9),
    medianprops=dict(color='black', linewidth=0.9),
    whiskerprops=dict(color='black', linewidth=0.8),
    capprops=dict(color='black', linewidth=0.8)
)
for patch in bp['boxes']:
    patch.set_facecolor('none')

# Scatter points (no mean lines)
rng = np.random.default_rng(42)
for j, vals in enumerate(sim_data):
    if len(vals) == 0:
        continue
    xj = rng.normal(loc=positions[j], scale=0.06, size=len(vals))
    ax.scatter(xj, vals, s=24, alpha=0.9, edgecolors='none',
               color=dist_colors.get(dist_order[j], 'black'))

# Stars if dissimilarity significant after Bonferroni
offset = (ax.get_ylim()[1] - ax.get_ylim()[0]) * 0.05
for j, (vals, sig) in enumerate(zip(sim_data, rej)):
    if sig and len(vals) > 0:
        y = 0.98
        ax.text(positions[j], y, "*", ha='center', va='bottom',
                fontsize=14, fontweight='bold')

ax.set_xticks(positions)
ax.set_xticklabels(dist_order, fontsize=10)
ax.set_ylabel('similarity score (a.u.)', fontsize=11)
ax.set_title('fix vs. mix similarity by distribution', fontsize=12, weight='bold')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.tight_layout()

# Save
fig.savefig(output_dir / "arrowup_percentage_fix_mix_participants_similarity.pdf",
            bbox_inches="tight", transparent=True)
fig.savefig(output_dir / "arrowup_percentage_fix_mix_participants_similarity.svg",
            dpi=1200, bbox_inches="tight", transparent=True)


## dissimilarity

In [24]:
# Models and custom colors
model_names = ['greedy', 'softmax', 'dualQ', 'risk-sensitive']
custom_colors = {
    'greedy':      '#56B4E9',
    'softmax':     '#009E73',
    'dualQ':       "#CEC10F",
    'risk-sensitive': "#065B8D"
}
distributions = ['uniform', 'low', 'high']

# -----------------------------------------------
# Collect distances separately by distribution
df_list = []

for dist in distributions:
    for model in model_names:
        p_vec = param_lookup.get((dist, 'participants'))
        m_vec = param_lookup.get((dist, model))
        if p_vec is not None and m_vec is not None:
            dist_val = np.linalg.norm(p_vec - m_vec)
            df_list.append({
                'Distribution': dist,
                'Model': model,
                'Distance_to_Participant': dist_val
            })

df_all = pd.DataFrame(df_list)

# -----------------------------------------------
# Plot: one subplot per distribution

title_colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

fig, axes = plt.subplots(1, 3, figsize=(8, 4), dpi=130, sharey=True)

for i, dist in enumerate(distributions):
    ax = axes[i]
    plot_data = df_all[df_all['Distribution'] == dist].sort_values('Distance_to_Participant', ascending=False)
    sns.barplot(
        data=plot_data,
        x='Model',
        y='Distance_to_Participant',
        palette=[custom_colors[m] for m in plot_data['Model']],
        ax=ax,
        ci=None
    )
    ax.set_title(f"{dist}", fontsize=11, fontweight='bold')
    ax.set_xlabel("model", fontsize=10)
    if i == 0:
        ax.set_ylabel("distance to participant", fontsize=10)
    else:
        ax.set_ylabel("")
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params(axis='x', rotation=30)



plt.suptitle("model distance to participants\n in arrow up percentage", fontsize=13, fontweight='bold')
plt.tight_layout(rect=[0, 0, 1, 1])

# -----------------------------------------------


# Save
# bar_pdf_path = output_dir / "arrowup_percentage_distance.pdf"
# bar_svg_path = output_dir / "arrowup_percentage_distance.svg"

# fig.savefig(bar_pdf_path)
# fig.savefig(bar_svg_path, format="svg", dpi=1200, bbox_inches="tight", transparent=True)
# plt.show()

# separate x0, k, L

In [25]:

# Sigmoid function
def sigmoid3(x, L, x0, k):
    return L / (1 + np.exp(-k * (x - x0)))

# -----------------------------------------------
# Distribution labels
distributions = ['uniform', 'low', 'high']


models = [
    (df_participants, 'participant', 'choice'),
    (df_greedy,       'greedy',      'model_choices'),
    (df_softmax,      'softmax',     'model_choices'),
    (df_dualQ,        'dualQ',       'model_choices'),
    (df_rs,           'risk-sensitive', 'model_choices'),
]

# -----------------------------------------------

def fit_subject_distribution(df, dist, choice_col):
    sub_df = df[df['distribution'] == dist].copy()
    if sub_df.empty:
        return None

    if choice_col != 'choice':
        sub_df[choice_col] = sub_df[choice_col].map({1: 'arrowup', 0: 'arrowdown'})

    sub_df['group'] = sub_df['myCard'].astype(str)
    group_counts = sub_df.groupby(['group', choice_col], observed=True)[choice_col].count().unstack(fill_value=0)
    group_totals = group_counts.sum(axis=1)
    group_percentages = (group_counts.T / group_totals).T 
    arrow_up = group_percentages.get('arrowup', pd.Series(0, index=group_counts.index)).values

    x = np.arange(len(arrow_up))

    # Force fit even with flat or limited variation
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", OptimizeWarning)
        try:
            # Fallback to small slope if flat
            p0 = [max(arrow_up), np.median(x), 0.1]  # start with gentle slope
            popt, _ = curve_fit(sigmoid3, x, arrow_up, p0, maxfev=10000, bounds=([0, 0, -10], [150, 10, 10]))
            return popt  # L, x0, k
        except (RuntimeError, ValueError):
            return None


# -----------------------------------------------
# Main loop to fit sigmoids per subject per model
fit_dfs = {}

for subject_dfs, label, choice_col in models:
    fit_rows = []
    print(f"--- Processing model: {label} ---")

    for i, df in enumerate(subject_dfs):
        for dist in distributions:
            popt = fit_subject_distribution(df, dist, choice_col)
            if popt is not None:
                fit_rows.append({
                    'subject_idx': i,
                    'distribution': dist,
                    'L': popt[0],
                    'x0': popt[1],
                    'k': popt[2],
                })
            else:
                print(f"    [Skipped] subject {i}, dist {dist} — fit failed or not enough variation")

    if fit_rows:
        fit_df = pd.DataFrame(fit_rows)
        fit_dfs[f'sigmoid_fit_{label}'] = fit_df
    else:
        print(f"No successful fits for model: {label}")

# -----------------------------------------------

sigmoid_fit_participant = fit_dfs.get('sigmoid_fit_participant')
sigmoid_fit_greedy = fit_dfs.get('sigmoid_fit_greedy')
sigmoid_fit_softmax = fit_dfs.get('sigmoid_fit_softmax')
sigmoid_fit_dualQ = fit_dfs.get('sigmoid_fit_dualQ')
sigmoid_fit_rs = fit_dfs.get('sigmoid_fit_risk-sensitive')






#########################################################################################################################################################


# -----------------------------------------------
def compute_model_similarity(part_df, model_df):
    """
    Return a DataFrame whose rows = subjects and columns = distributions
    (uniform, low, high).  Each cell contains either RMSE (lower = better)
    or cosine similarity (higher = better) between the participant’s and
    model’s fitted sigmoid curves evaluated at card numbers 1–9.
    """
    dists    = ['uniform', 'low', 'high']
    subjects = sorted(part_df['subject_idx'].unique())

    # empty matrix to fill
    sim_mat = pd.DataFrame(index=subjects, columns=dists, dtype=float)

    cards = np.arange(1, 10)          # evaluate sigmoid at 1–9

    for subj in subjects:
        for dist in dists:
            p_row = part_df.query('subject_idx == @subj and distribution == @dist')
            m_row = model_df.query('subject_idx == @subj and distribution == @dist')

            if p_row.empty or m_row.empty:
                sim_mat.loc[subj, dist] = np.nan
                continue

            p_hat = sigmoid3(cards, *p_row[['L', 'x0', 'k']].iloc[0].values)
            m_hat = sigmoid3(cards, *m_row[['L', 'x0', 'k']].iloc[0].values)

            val = np.sqrt(mean_squared_error(p_hat, m_hat))        # smaller = better
            
            sim_mat.loc[subj, dist] = val

    return sim_mat
# -----------------------------------------------

# recompute similarities as DataFrames
similarity_greedy  = compute_model_similarity(sigmoid_fit_participant, sigmoid_fit_greedy)
similarity_softmax = compute_model_similarity(sigmoid_fit_participant, sigmoid_fit_softmax)
similarity_dualQ   = compute_model_similarity(sigmoid_fit_participant, sigmoid_fit_dualQ)
similarity_rs      = compute_model_similarity(sigmoid_fit_participant, sigmoid_fit_rs)

# ------------------------------------------------
# downstream code remains unchanged
model_names   = ['greedy', 'softmax', 'dualQ', 'risk-sensitive']
similarity_dfs = [similarity_greedy, similarity_softmax, similarity_dualQ, similarity_rs]

long_data = []
for model_name, sim_df in zip(model_names, similarity_dfs):
    for subj in sim_df.index:
        for dist in ['uniform', 'low', 'high']:
            sim_val = sim_df.loc[subj, dist]
            long_data.append({
                'Subject': subj,
                'Distribution': dist,
                'Similarity': sim_val,
                'Model': model_name
            })

df_all = pd.DataFrame(long_data)


--- Processing model: participant ---
--- Processing model: greedy ---
--- Processing model: softmax ---
--- Processing model: dualQ ---
--- Processing model: risk-sensitive ---


In [26]:

custom_colors = {
    'greedy':         '#56B4E9',
    'softmax':        '#009E73',
    'dualQ':          "#CEC10F",
    'risk-sensitive': "#065B8D"
}

# -------------------------------------------------
# Plot
distributions = ['uniform', 'low', 'high']
fig, axes = plt.subplots(1, 3, figsize=(14, 6), sharey=True)

for i, dist in enumerate(distributions):
    ax = axes[i]
    sub_df = df_all[df_all['Distribution'] == dist]
    models = sub_df['Model'].unique()

    # Calculate mean dissimilarity per model and sort
    model_means = sub_df.groupby('Model')['Similarity'].mean()
    sorted_models = model_means.sort_values(ascending=False).index.tolist()

    # Reorder data and positions based on sorted models
    data = [sub_df[sub_df['Model'] == model]['Similarity'].values for model in sorted_models]
    positions = np.arange(len(sorted_models))

    # Boxplot (edges only, no fill)
# Boxplot (edges only, no fill)
    bp = ax.boxplot(
        data,
        positions=positions,
        patch_artist=True,
        widths=0.5,
        showfliers=False,
        boxprops=dict(facecolor='none', edgecolor='black', linewidth=0.5),
        medianprops=dict(color='black', linewidth=0.5),
        capprops=dict(color='black', linewidth=0.5),
        whiskerprops=dict(color='black', linewidth=0.5)
    )

    for patch in bp['boxes']:
        patch.set_facecolor('none')

    # Scatter points
    for j, model in enumerate(sorted_models):
        y_vals = sub_df[sub_df['Model'] == model]['Similarity'].values
        x_vals = np.random.normal(loc=j, scale=0.08, size=len(y_vals))  # jitter
        ax.scatter(
            x_vals,
            y_vals,
            s=18,
            alpha=0.8,
            edgecolors='none',
            linewidths=0.3,
            color=custom_colors[model]
        )

    # Clean axes and labels
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_title(f'{dist}', fontsize=13)
    if i == 0:
        ax.set_ylabel('dissimilarity score')
    ax.set_xticks(positions)
    ax.set_xticklabels(sorted_models, rotation=30)
    ax.set_xlabel('')

    # -------------------------------------------------
    # Run ANOVA
    f_stat, p_val = f_oneway(*data)
    print(f"\n[ANOVA] {dist}: F = {f_stat:.4f}, p = {p_val:.4f}")

    # -------------------------------------------------
    # Tukey HSD for post‑hoc pairwise comparisons
    tukey = pairwise_tukeyhsd(
        endog=sub_df['Similarity'],
        groups=sub_df['Model'],
        alpha=0.05
    )
    print(tukey)

    base_offset = 0.55
    step_offset = 0.08
    bar_height  = 0.03
    bar_count   = 0

    # Add stars between significant pairs
    for res in tukey.summary().data[1:]:
        group1, group2, _, p_adj, _, _, reject = res
        if reject:
            idx1 = sorted_models.index(group1)
            idx2 = sorted_models.index(group2)
            x1, x2 = positions[idx1], positions[idx2]

            y = base_offset + bar_count * step_offset

            ax.plot([x1, x1, x2, x2],
                    [y, y + bar_height, y + bar_height, y],
                    lw=0.7, color='black')

            stars = '***' if p_adj < 0.001 else ('**' if p_adj < 0.01 else '*')
            ax.text((x1 + x2) / 2,
                    y + bar_height + 0.005,
                    stars,
                    ha='center',
                    va='bottom',
                    fontsize=15)

            bar_count += 1

    ylim_low, ylim_high = ax.get_ylim()
    if y + bar_height + 0.05 > ylim_high:
        ax.set_ylim(ylim_low, y + bar_height + 0.05)

plt.tight_layout()
plt.suptitle(
    "rmse of fitted sigmoid curves on arrowup percentage\n"
    "between models and participants",
    fontsize=15,
    y=1.05
)

# -------------------------------------------------
# Save figure
# bar_pdf_path = output_dir / "arrowup_percentage_dissimilarity.pdf"
# bar_svg_path = output_dir / "arrowup_percentage_dissimilarity.svg"
# fig.savefig(bar_pdf_path, bbox_inches="tight", transparent=True)
# fig.savefig(bar_svg_path, format="svg", dpi=1200, bbox_inches="tight", transparent=True)
# plt.show()



[ANOVA] uniform: F = 48.4800, p = 0.0000
        Multiple Comparison of Means - Tukey HSD, FWER=0.05         
    group1         group2     meandiff p-adj   lower   upper  reject
--------------------------------------------------------------------
         dualQ         greedy   0.0284 0.1529 -0.0065  0.0633  False
         dualQ risk-sensitive  -0.1214    0.0 -0.1562 -0.0865   True
         dualQ        softmax  -0.0052 0.9806   -0.04  0.0297  False
        greedy risk-sensitive  -0.1498    0.0 -0.1846 -0.1149   True
        greedy        softmax  -0.0336 0.0637 -0.0685  0.0013  False
risk-sensitive        softmax   0.1162    0.0  0.0813   0.151   True
--------------------------------------------------------------------

[ANOVA] low: F = 26.1581, p = 0.0000
        Multiple Comparison of Means - Tukey HSD, FWER=0.05         
    group1         group2     meandiff p-adj   lower   upper  reject
--------------------------------------------------------------------
         dualQ         

Text(0.5, 1.05, 'rmse of fitted sigmoid curves on arrowup percentage\nbetween models and participants')

# arrow up down percentage fix and mix separated dissimilarities

In [27]:

# Sigmoid
def sigmoid3(x, L, x0, k):
    return L / (1 + np.exp(-k * (x - x0)))

# -------------------------------------------------
# Config
distributions = ['uniform', 'low', 'high']
blocks        = ['fix', 'mix']  # row 1, row 2

# Models and colors (matches your code)
models = [
    (df_participants, 'participant',    'choice'),
    (df_greedy,       'greedy',         'model_choices'),
    (df_softmax,      'softmax',        'model_choices'),
    (df_rs,           'risk-sensitive', 'model_choices'),
]

custom_colors = {
    'greedy':         '#56B4E9',
    'softmax':        '#009E73',
    'risk-sensitive': "#065B8D"
}

# -------------------------------------------------
# Fit a single subject × distribution × block
def fit_subject_distribution_block(df, dist, choice_col, block_label):
    """
    Fit sigmoid to a single subject's arrow-up proportion by myCard,
    within a specific distribution and block ('fix' or 'mix').
    Returns popt = (L, x0, k) or None.
    """
    d = df[df['block_type'] == block_label]
    if d.empty:
        return None

    sub_df = d[d['distribution'] == dist].copy()
    if sub_df.empty:
        return None

    if choice_col != 'choice':
        sub_df[choice_col] = sub_df[choice_col].map({1: 'arrowup', 0: 'arrowdown'})

    sub_df['group'] = sub_df['myCard'].astype(str)

    gc = sub_df.groupby(['group', choice_col], observed=True)[choice_col].count().unstack(fill_value=0)
    if gc.empty:
        return None

    gt = gc.sum(axis=1)
    # Guard against zero totals (just in case)
    gt = gt.replace(0, np.nan)

    gp = (gc.T / gt).T
    if 'arrowup' not in gp.columns:
        return None

    arrow_up = gp['arrowup'].fillna(0).values
    if arrow_up.size < 3:
        return None

    x = np.arange(len(arrow_up))
    try:
        p0 = [max(arrow_up), np.median(x), 0.1]  # mild slope default
        popt, _ = curve_fit(
            sigmoid3, x, arrow_up,
            p0=p0, maxfev=20000,
            bounds=([0, 0, -10], [1.5, 10, 10])  # L bound to 1.5 to be safe
        )
        return popt
    except Exception:
        return None

# -------------------------------------------------
# Fit sigmoids for all subjects, models, distributions, by block
def fit_all_by_block(block_label):
    fit_dfs = {}
    for subject_dfs, label, choice_col in models:
        rows = []
        for i, df in enumerate(subject_dfs):
            for dist in distributions:
                popt = fit_subject_distribution_block(df, dist, choice_col, block_label)
                if popt is not None:
                    rows.append({
                        'subject_idx': i,
                        'distribution': dist,
                        'L':  popt[0],
                        'x0': popt[1],
                        'k':  popt[2],
                    })
        if rows:
            fit_dfs[f'sigmoid_fit_{label}'] = pd.DataFrame(rows)
        else:
            fit_dfs[f'sigmoid_fit_{label}'] = pd.DataFrame(
                columns=['subject_idx', 'distribution', 'L', 'x0', 'k']
            )
    return fit_dfs

# -------------------------------------------------
# Compute RMSE similarity matrix (subjects × distributions) for a model vs participants
def compute_model_similarity(part_df, model_df):
    dists    = ['uniform', 'low', 'high']
    subjects = sorted(part_df['subject_idx'].unique())
    sim_mat  = pd.DataFrame(index=subjects, columns=dists, dtype=float)
    cards    = np.arange(1, 10)  # evaluate at 1–9

    for subj in subjects:
        for dist in dists:
            p_row = part_df.query('subject_idx == @subj and distribution == @dist')
            m_row = model_df.query('subject_idx == @subj and distribution == @dist')
            if p_row.empty or m_row.empty:
                sim_mat.loc[subj, dist] = np.nan
                continue
            p_hat = sigmoid3(cards, *p_row[['L', 'x0', 'k']].iloc[0].values)
            m_hat = sigmoid3(cards, *m_row[['L', 'x0', 'k']].iloc[0].values)
            sim_mat.loc[subj, dist] = np.sqrt(mean_squared_error(p_hat, m_hat))
    return sim_mat

# -------------------------------------------------
# Run fits for both blocks, then compute RMSEs
fits_fix = fit_all_by_block('fix')
fits_mix = fit_all_by_block('mix')

part_fix = fits_fix['sigmoid_fit_participant']
part_mix = fits_mix['sigmoid_fit_participant']

block_sims = []
for block_label, fits in [('fix', fits_fix), ('mix', fits_mix)]:
    part_df = fits['sigmoid_fit_participant']

    # Skip if no participant fits
    if part_df.empty:
        continue

    for model_name, key in [
        ('greedy',         'sigmoid_fit_greedy'),
        ('softmax',        'sigmoid_fit_softmax'),
        ('dualQ',          'sigmoid_fit_dualQ'),
        ('risk-sensitive', 'sigmoid_fit_risk-sensitive'),
    ]:
        model_df = fits.get(key, pd.DataFrame())
        if model_df.empty:
            continue
        sim_df = compute_model_similarity(part_df, model_df)  # subjects × dist
        # long format with Block label
        for subj in sim_df.index:
            for dist in distributions:
                block_sims.append({
                    'Block': block_label,
                    'Subject': subj,
                    'Distribution': dist,
                    'Similarity': sim_df.loc[subj, dist],
                    'Model': model_name
                })

df_all = pd.DataFrame(block_sims).dropna(subset=['Similarity'])

# -------------------------------------------------
# Plot: 2 rows (fix/mix) × 3 cols (distributions), box + jitter, ANOVA + Tukey per cell
fig, axes = plt.subplots(2, 3, figsize=(10, 8), sharey=True)

for r, block_label in enumerate(blocks):
    for c, dist in enumerate(distributions):
        ax = axes[r, c]
        sub_df = df_all[(df_all['Block'] == block_label) & (df_all['Distribution'] == dist)].copy()
        if sub_df.empty:
            ax.set_title(f'{dist} ({block_label})')
            ax.set_xticks([]); ax.set_yticks([])
            ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
            continue

        # Order models by mean dissimilarity (descending)
        model_means = sub_df.groupby('Model')['Similarity'].mean().sort_values(ascending=False)
        sorted_models = model_means.index.tolist()

        data = [sub_df[sub_df['Model'] == m]['Similarity'].values for m in sorted_models]
        positions = np.arange(len(sorted_models))

        # Box (outline only)
        bp = ax.boxplot(
            data,
            positions=positions,
            patch_artist=True,
            widths=0.5,
            showfliers=False,
            boxprops=dict(facecolor='none', edgecolor='black', linewidth=0.6),
            medianprops=dict(color='black', linewidth=0.6),
            capprops=dict(color='black', linewidth=0.6),
            whiskerprops=dict(color='black', linewidth=0.6)
        )
        for patch in bp['boxes']:
            patch.set_facecolor('none')

        # Jittered points
        for j, m in enumerate(sorted_models):
            y_vals = sub_df[sub_df['Model'] == m]['Similarity'].values
            x_vals = np.random.normal(loc=j, scale=0.08, size=len(y_vals))
            ax.scatter(
                x_vals, y_vals, s=18, alpha=0.85,
                edgecolors='none', linewidths=0.3,
                color=custom_colors.get(m, 'black')
            )

        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_title(f'{dist} ({block_label})', fontsize=12, weight='bold')
        if c == 0:
            ax.set_ylabel('dissimilarity score (a.u.)', fontsize=11)
        ax.set_xticks(positions)
        ax.set_xticklabels(sorted_models, rotation=30)
        ax.set_xlabel('')

        # -------- Stats: ANOVA + Tukey per cell --------

        try:
            f_stat, p_val = f_oneway(*data)
            print(f"[ANOVA] {dist} | {block_label}: F = {f_stat:.4f}, p = {p_val:.4f}")
            tukey = pairwise_tukeyhsd(endog=sub_df['Similarity'], groups=sub_df['Model'], alpha=0.05)
            print(tukey)

            # --- Adjusted parameters ---
            ylim_low, ylim_high = ax.get_ylim()
            base_offset = ylim_high * 0.90          # start higher (90% of axis height)
            step_offset = (ylim_high - ylim_low) * 0.08   # more vertical spacing between bars
            bar_height  = (ylim_high - ylim_low) * 0.02   # taller bar

            bar_count = 0
            for res in tukey.summary().data[1:]:
                g1, g2, _, p_adj, _, _, reject = res
                if reject and g1 in sorted_models and g2 in sorted_models:
                    idx1 = sorted_models.index(g1)
                    idx2 = sorted_models.index(g2)
                    x1, x2 = positions[idx1], positions[idx2]
                    y = base_offset + bar_count * step_offset

                    ax.plot([x1, x1, x2, x2],
                            [y, y + bar_height, y + bar_height, y],
                            lw=0.9, color='black')

                    stars = '***' if p_adj < 0.001 else ('**' if p_adj < 0.01 else '*')
                    ax.text((x1 + x2) / 2,
                            y + bar_height + 0.01,
                            stars,
                            ha='center',
                            va='bottom',
                            fontsize=13)
                    bar_count += 1

            # Extend ylim if needed
            if bar_count > 0 and y + bar_height + 0.05 > ylim_high:
                ax.set_ylim(ylim_low, y + bar_height + 0.08)

        except Exception as e:
            print(f"[Stats skipped] {dist} | {block_label}: {e}")



plt.suptitle(
    "model and participant response curve dissimilarity",
    fontsize=14, y=0.995
)
plt.tight_layout(rect=[0, 0, 1, 0.97])

# -------------------------------------------------
# Save
# output_dir must exist or be defined elsewhere; else comment out or set "."
fig.savefig(output_dir / "arrowup_percentage_fix_mix_dissimilarity.pdf", bbox_inches="tight", transparent=True)
fig.savefig(output_dir / "arrowup_percentage_fix_mix_dissimilarity.svg", format="svg", dpi=1200, bbox_inches="tight", transparent=True)


[ANOVA] uniform | fix: F = 34.9474, p = 0.0000
        Multiple Comparison of Means - Tukey HSD, FWER=0.05         
    group1         group2     meandiff p-adj   lower   upper  reject
--------------------------------------------------------------------
        greedy risk-sensitive  -0.1305    0.0 -0.1705 -0.0905   True
        greedy        softmax  -0.0188 0.5068 -0.0588  0.0212  False
risk-sensitive        softmax   0.1117    0.0  0.0717  0.1517   True
--------------------------------------------------------------------
[ANOVA] low | fix: F = 17.7695, p = 0.0000
        Multiple Comparison of Means - Tukey HSD, FWER=0.05         
    group1         group2     meandiff p-adj   lower   upper  reject
--------------------------------------------------------------------
        greedy risk-sensitive  -0.1067 0.0001 -0.1666 -0.0467   True
        greedy        softmax   0.0391 0.2731 -0.0209  0.0991  False
risk-sensitive        softmax   0.1458    0.0  0.0858  0.2058   True
-------------

# delta q and arrowRT

In [28]:
# Step 1: Build arrowRT values split by distribution
dist_names = ['uniform', 'low', 'high']
arrowRT_participants_by_dist = {dist: [] for dist in dist_names}

for df in df_participants:
    arrowRT_norm = np.array(df['arrowRT'], dtype=float) / np.mean(df['arrowRT'])
    distributions = df['distribution'].tolist()

    dist_split = {dist: [] for dist in dist_names}
    for rt, dist in zip(arrowRT_norm, distributions):
        dist_split[dist].append(rt)

    for dist in dist_names:
        arrowRT_participants_by_dist[dist].append(dist_split[dist])  # list of lists

# Convert to np.array for each distribution
for dist in dist_names:
    arrowRT_participants_by_dist[dist] = np.array([np.array(x) for x in arrowRT_participants_by_dist[dist]])


#######################################################################################################################################


def compute_delta_q_vals(df_model):
    # Initialize: 3 distributions × 9 card numbers
    dist_names = ['uniform', 'low', 'high']
    deltas_all = {
        dist: [[] for _ in range(9)] for dist in dist_names
    }

    for df in df_model:
        dist_trials = df['distribution'].tolist()
        q_val_trials = df['q_val'].tolist()

        # Create empty structure for this participant
        dist_deltas = {
            dist: [[] for _ in range(9)] for dist in dist_names
        }

        for dist_name, q_str in zip(dist_trials, q_val_trials):
            q_array = np.array(ast.literal_eval(q_str))  # shape: (9, trials, 2)
            delta = q_array[:, :, actions["arrowup"]] - q_array[:, :, actions["arrowdown"]]
            delta = np.abs(delta)  # absolute difference
            for card_idx in range(9):
                dist_deltas[dist_name][card_idx].append(delta[card_idx])  # list of arrays

        # Append this participant's data to global pool
        for dist in dist_names:
            for card_idx in range(9):
                deltas_all[dist][card_idx].append(dist_deltas[dist][card_idx])  # list of list of arrays

    # Convert each to np.array (participants × trials)
    return {
        dist: [np.array(card_list) for card_list in deltas_all[dist]]
        for dist in dist_names
    }

################################################################

def permutation_corr(x, y, n_permutations=1000, seed=42):
    rng = np.random.default_rng(seed)
    actual_corr, _ = spearmanr(x, y)
    permuted_corrs = np.zeros(n_permutations)
    
    for i in range(n_permutations):
        y_perm = rng.permutation(y)
        permuted_corrs[i], _ = spearmanr(x, y_perm)
    
    pval = np.mean(np.abs(permuted_corrs) >= np.abs(actual_corr))
    return actual_corr, pval

# List of models and their dataframes
model_dfs = [
    ('greedy', df_greedy),
    ('softmax', df_softmax),
    ('RS', df_rs)
]




In [29]:
fig = plt.figure(figsize=(5.5, 4))
gs = gridspec.GridSpec(nrows=1, ncols=4, width_ratios=[1, 1, 1, 0.05], wspace=0.3)
axes = [fig.add_subplot(gs[0, i]) for i in range(3)]
cbar_ax = fig.add_subplot(gs[0, 3])

for ax, (model_name, df_model) in zip(axes, model_dfs):
    delta_q_vals_by_dist = compute_delta_q_vals(df_model)

    correlation_means = np.zeros((9, 3))
    pvals_means = np.zeros((9, 3))

    for dist_idx, dist_name in enumerate(['uniform', 'low', 'high']):
        delta_q_vals = delta_q_vals_by_dist[dist_name]
        arrowRT_dist = arrowRT_participants_by_dist[dist_name]
        arrowRT_avg = np.mean(arrowRT_dist, axis=0)
        
        for card_idx in range(9):
            delta_card = delta_q_vals[card_idx][:][dist_idx]
            delta_card_mean = np.mean(delta_card, axis=1)

            corr, pval = permutation_corr(arrowRT_avg, delta_card_mean, n_permutations=1000)
            correlation_means[card_idx, dist_idx] = corr
            pvals_means[card_idx, dist_idx] = pval

    flat_pvals = pvals_means.flatten()
    _, pvals_corrected, _, _ = multipletests(flat_pvals, alpha=0.05, method='fdr_bh')
    pvals_corrected = pvals_corrected.reshape((9, 3))

    # Annotate with stars after correction
    annot = np.empty((9, 3), dtype=object)
    for i in range(9):
        for j in range(3):
            annot[i, j] = '*' if pvals_corrected[i, j] < 0.05 else ''

    sns.heatmap(
        correlation_means,
        cmap='coolwarm',
        annot=annot,
        fmt='',
        xticklabels=['uniform', 'low', 'high'],  
        yticklabels=[str(i) for i in range(1, 10)],
        linewidths=0.5,
        vmin=-0.4, vmax=0.4,
        cbar=(ax == axes[-1]),
        cbar_ax=cbar_ax if ax == axes[-1] else None,
        ax=ax
    )

    ax.set_title(model_name, fontsize=10, fontweight='bold')
    ax.set_xlabel("distribution", fontsize=10, fontweight='bold', rotation=45)
    ax.set_ylabel("card number" if ax == axes[0] else "")

plt.suptitle("|Q(up) - Q(down)| and RT correlation", fontsize=12, fontweight='bold')
plt.tight_layout(rect=[0, 0, 1, 0.95])

# fig.savefig(output_dir / "correlation_avg_qVal_arrow_RT.pdf")
# fig.savefig(output_dir / "correlation_avg_qVal_arrow_RT.svg", format="svg", dpi=1200, bbox_inches="tight", transparent=True)


# delta q vals changes

In [30]:
def compute_delta_q_vals_no_abs(df_model):
    # Initialize: 3 distributions × 9 card numbers
    dist_names = ['uniform', 'low', 'high']
    deltas_all = {
        dist: [[] for _ in range(9)] for dist in dist_names
    }

    for df in df_model:
        dist_trials = df['distribution'].tolist()
        q_val_trials = df['q_val'].tolist()

        # Create empty structure for this participant
        dist_deltas = {
            dist: [[] for _ in range(9)] for dist in dist_names
        }

        for dist_name, q_str in zip(dist_trials, q_val_trials):
            q_array = np.array(ast.literal_eval(q_str))
            delta = q_array[:, :, actions["arrowup"]] - q_array[:, :, actions["arrowdown"]]
            for card_idx in range(9):
                dist_deltas[dist_name][card_idx].append(delta[card_idx])  # list of arrays

        # Append this participant's data to global pool
        for dist in dist_names:
            for card_idx in range(9):
                deltas_all[dist][card_idx].append(dist_deltas[dist][card_idx])  # list of list of arrays

    # Convert each to np.array (participants × trials)
    return {
        dist: [np.array(card_list) for card_list in deltas_all[dist]]
        for dist in dist_names
    }

#######################################################################################################################################

fig, axs = plt.subplots(3, 3, figsize=(7, 7), sharex=True, sharey=True)
dist_names = ['uniform', 'low', 'high']
model_names = ['greedy', 'softmax', 'RS']
colors = sns.color_palette("tab10", 9)  # one color per card

for row_idx, (model_name, df_model) in enumerate(model_dfs):
    delta_q_vals_by_dist = compute_delta_q_vals_no_abs(df_model)

    for col_idx, dist_name in enumerate(dist_names):
        ax = axs[row_idx, col_idx]
        delta_q_vals = delta_q_vals_by_dist[dist_name]  # list of 9 arrays, shape (n_participants, n_trials)

        for card_idx in range(9):
            delta = delta_q_vals[card_idx]  # shape: (n_participants, n_trials)
            mean_over_participants = np.mean(delta, axis=0)  # mean across participants per trial
            mean_over_participants = mean_over_participants[:, col_idx]
            ax.plot(mean_over_participants, label=f'{card_idx + 1}', color=colors[card_idx], linewidth=1)

        ax.set_ylim(-0.6, 0.6)
        ax.set_yticks([-0.6, 0, 0.6])
        ax.set_xticks(np.arange(0, 100, 10))

        # Remove top and right spines
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        if row_idx == 2:
            ax.set_xlabel("Trial", fontsize=10, fontweight='bold')
        if col_idx == 0:
            ax.set_ylabel(model_name, fontsize=11, fontweight='bold')
        if row_idx == 0:
            ax.set_title(dist_name, fontsize=11, fontweight='bold')

# Create legend text in colors (no lines)
legend_labels = [plt.Text(text=str(i+1), color=colors[i], fontsize=9) for i in range(9)]
legend_fig = fig.add_subplot(111, frameon=False)
legend_fig.axis('off')
legend_y = 0.92  # y-position for the row
x_start = 0.1    # starting x-position
x_step = 0.08    # spacing between labels

for i in range(9):
    fig.text(x_start + i * x_step, legend_y, str(i + 1), color=colors[i], fontsize=9)

# Add main title
plt.suptitle("ΔQ (up - down) over trials per card, model, and distribution", fontsize=14, fontweight='bold', y=0.96)

plt.tight_layout(rect=[0, 0, 1, 0.91])  # adjusted for new legend space

# Save figure
# fig.savefig(output_dir / "deltaQ_mean.pdf")
# fig.savefig(output_dir / "deltaQ_mean.svg", format="svg", dpi=1200, bbox_inches="tight", transparent=True)



In [31]:
fig, axs = plt.subplots(1, 3, figsize=(9, 4), sharey=True)  # 1 row, 3 columns
dist_names = ['uniform', 'low', 'high']
model_names = ['greedy', 'softmax', 'RS']
card_numbers = np.arange(1, 10)
title_colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

for col_idx, (model_name, df_model) in enumerate(model_dfs):
    ax = axs[col_idx]
    delta_q_vals_by_dist = compute_delta_q_vals_no_abs(df_model)

    for dist_name in dist_names:
        delta_q_vals = delta_q_vals_by_dist[dist_name]  # list of 9 arrays: (n_participants, n_trials)

        mean_deltas = []
        sem_deltas = []

        for card_idx in range(9):
            delta = delta_q_vals[card_idx]  # shape: (n_participants, n_trials)
            card_means = np.mean(delta, axis=1)  # mean per participant
            mean_deltas.append(np.mean(card_means))  # group mean
            sem_deltas.append(np.std(card_means, ddof=1) / np.sqrt(len(card_means)))  # SEM

        ax.errorbar(
            card_numbers,
            mean_deltas,
            yerr=sem_deltas,
            fmt='-o',
            color=title_colors[dist_name],
            label=dist_name,
            capsize=2,         # shorter caps
            capthick=0.6,        # thinner caps
            elinewidth=0.6,      # thinner error bars
            linewidth=0.6,       # thinner line connecting points
            markersize=2
        )

    # Axis formatting
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    ax.set_ylim(-0.5, 0.5)
    ax.set_yticks([-0.5, 0, 0.5])
    ax.set_xticks(card_numbers)
    ax.set_xlabel("Card number", fontsize=10, fontweight='bold')
    ax.set_title(model_name, fontsize=11, fontweight='bold')

axs[0].set_ylabel("ΔQ (up - down)", fontsize=11, fontweight='bold')

# Common title and legend
fig.suptitle("ΔQ (up - down) means per card across distributions \n(don't) forget legend", fontsize=14, fontweight='bold')


plt.tight_layout(rect=[0, 0, 1, 0.98])

# fig.savefig(output_dir / "deltaQ_mean_numbers_dist.pdf")
# fig.savefig(output_dir / "deltaQ_mean_numbers_dist.svg", format="svg", dpi=1200, bbox_inches="tight", transparent=True)


# participant choice prediction using delta Q

In [32]:
###########################################################fixed effects model################################################################


def run_mixed_logistic(df_reg):
    # Set category order so 'uniform' is dropped
    df = df_reg.copy()
    df['distribution'] = pd.Categorical(df['distribution'], categories=['uniform', 'high', 'low'], ordered=True)

    # Dummy-code the distribution column, dropping 'uniform'
    df = pd.get_dummies(df, columns=['distribution'], drop_first=True)
    dist_cols = [col for col in df.columns if col.startswith('distribution_')]

    # Select and coerce predictors
    predictors = ['delta_q', 'log_trial'] + dist_cols
    X = df[predictors].astype(float)
    X = sm.add_constant(X)
    y = pd.to_numeric(df['choice'], errors='coerce')
    valid_idx = X.notnull().all(axis=1) & y.notnull()
    X = X[valid_idx]
    y = y[valid_idx]
    groups = df['participant'][valid_idx]

    # Fit model
    model = sm.Logit(y, X)
    result = model.fit(disp=False)

    # Clustered standard errors
    cov = cov_cluster(result, groups)
    params = result.params
    bse = np.sqrt(np.diag(cov))

    # Compute z-scores and raw p-values
    z_vals = params / bse
    p_vals = 2 * (1 - norm.cdf(np.abs(z_vals)))

    # Multiple comparisons correction (FDR - Benjamini-Hochberg)
    p_vals_fdr = multipletests(p_vals, method='fdr_bh')[1]

    # Create summary table
    summary_table = pd.DataFrame({
        'coef': params,
        'std_err (clustered)': bse,
        'z': z_vals,
        'p': p_vals_fdr
    })

    return result, summary_table





########################################################### mixed effects model################################################################
# def run_mixed_logistic(df_reg):
#     df = df_reg.copy()

#     # encode distribution dummies (uniform is reference)
#     df['distribution'] = pd.Categorical(
#         df['distribution'],
#         categories=['uniform', 'high', 'low'],
#         ordered=True
#     )
#     df = pd.get_dummies(df, columns=['distribution'], drop_first=True)

#     # keep only relevant, non-missing columns
#     keep = ['choice', 'delta_q', 'log_trial',
#             'distribution_high', 'distribution_low', 'participant']
#     df = df[keep].dropna()

#     # ---- model spec ----
#     fe_formula = ("choice ~ delta_q + log_trial + "
#                   "distribution_high + distribution_low")        # fixed effects
#     re_formula = {"participant": "1 + delta_q"}                  # random effects

#     # fit variational-Bayes GLMM
#     model  = BinomialBayesMixedGLM.from_formula(fe_formula,
#                                                 re_formula,
#                                                 data=df)
#     result = model.fit_vb()                                      # seconds–minutes


#     fe_names = model.exog_names              # predictor names
#     fe_mean  = result.fe_mean                # posterior means
#     fe_sd    = result.fe_sd                  # posterior std. devs.

#     z_vals   = fe_mean / fe_sd
#     p_vals   = 2 * norm.sf(abs(z_vals))      # two-tailed p

#     # Benjamini–Hochberg FDR correction
#     _, p_adj, _, _ = multipletests(p_vals, alpha=0.05, method='fdr_bh')

#     summary_df = pd.DataFrame({
#         'predictor' : fe_names,
#         'coef'      : fe_mean,
#         'std_err'   : fe_sd,
#         'z'         : z_vals,
#         'pval'      : p_vals,
#         'pval_adj'  : p_adj
#     })

#     # attach for convenient access
#     result.summary_df = summary_df
#     result.pval_adj   = dict(zip(fe_names, p_adj))

#     return result



# def tidy_bayes_glmm(fit, model_name):
#     names = fit.model.exog_names                     # coefficient labels

#     coef = pd.Series(fit.fe_mean, index=names, name="coef")
#     se   = pd.Series(fit.fe_sd,   index=names, name="std_err (clustered)")

#     z  = (coef / se).rename("z")                     # ensure Series, not ndarray
#     p  = pd.Series(2 * (1 - norm.cdf(np.abs(z))), index=names, name="p")

#     out = pd.concat([coef, se, z, p], axis=1).reset_index()
#     out = out.rename(columns={"index": "predictor"})
    
#     # match the predictor names your plotting code expects
#     out["predictor"] = out["predictor"].replace({
#         "Intercept": "const",
#         "distribution_high[T.True]": "distribution_high",
#         "distribution_low[T.True]":  "distribution_low",
#     })
#     out["model"] = model_name
#     return out




###########################################################################################################################


def build_delta_q_regression_df_from_model(df_model, model_name, participant_id):
    rows = []
    dist_map = {'uniform': 0, 'low': 1, 'high': 2}

    for trial_idx, row in df_model.iterrows():
        q_array = np.array(ast.literal_eval(row['q_val']))  # shape: (9, 3, 2)
        card_idx = int(row['myCard']) - 1  # zero-index card number
        dist_idx = dist_map[row['distribution']]  # map dist to 0/1/2

        q_up = q_array[card_idx, dist_idx, actions["arrowup"]]
        q_down = q_array[card_idx, dist_idx, actions["arrowdown"]]
        delta_q = q_up - q_down

        rows.append({
            'participant': f'p{participant_id + 1}',
            'trial': trial_idx,
            'log_trial': np.log(trial_idx + 1),
            'choice': row['participant_choices'],
            'distribution': row['distribution'],
            'card_number': row['myCard'],
            'delta_q': delta_q,
            'model': model_name
        })

    return pd.DataFrame(rows)



################################################################################################################################
reg_dfs_greedy = []

for pid, df_model in enumerate(df_greedy):
    df = build_delta_q_regression_df_from_model(df_model, model_name='greedy', participant_id=pid)
    reg_dfs_greedy.append(df)

df_reg_greedy = pd.concat(reg_dfs_greedy, ignore_index=True)


reg_dfs_softmax = []
for pid, df_model in enumerate(df_softmax):
    df = build_delta_q_regression_df_from_model(df_model, model_name='softmax', participant_id=pid)
    reg_dfs_softmax.append(df)
df_reg_softmax = pd.concat(reg_dfs_softmax, ignore_index=True)


reg_dfs_rs = []
for pid, df_model in enumerate(df_rs):
    df = build_delta_q_regression_df_from_model(df_model, model_name='RS', participant_id=pid)
    reg_dfs_rs.append(df)
df_reg_rs = pd.concat(reg_dfs_rs, ignore_index=True)

###################################################fix effects model########################################################


result_greedy, summary_greedy = run_mixed_logistic(df_reg_greedy)
result_softmax, summary_softmax = run_mixed_logistic(df_reg_softmax)
result_rs, summary_rs = run_mixed_logistic(df_reg_rs)

summary_greedy['model'] = 'greedy'
summary_softmax['model'] = 'softmax'
summary_rs['model'] = 'RS'

# Combine summaries
summary_all = pd.concat([summary_greedy, summary_softmax, summary_rs])
summary_all = summary_all.reset_index().rename(columns={'index': 'predictor'})

# Reorder predictors
predictor_order = ['delta_q', 'distribution_low', 'distribution_high', 'log_trial', 'const']
summary_all['predictor'] = pd.Categorical(summary_all['predictor'], categories=predictor_order, ordered=True)
summary_all = summary_all.sort_values(['predictor', 'model']).reset_index(drop=True)
###########################################################################################################################



################################################### mix effects model ########################################################


# fit_greedy  = run_mixed_logistic(df_reg_greedy)
# fit_softmax = run_mixed_logistic(df_reg_softmax)
# fit_rs      = run_mixed_logistic(df_reg_rs)


# summary_greedy  = tidy_bayes_glmm(fit_greedy,  "greedy")
# summary_softmax = tidy_bayes_glmm(fit_softmax, "softmax")
# summary_rs      = tidy_bayes_glmm(fit_rs,      "RS")

# summary_all = (
#     pd.concat([summary_greedy, summary_softmax, summary_rs], ignore_index=True)
#       .assign(predictor=lambda d: pd.Categorical(
#           d["predictor"],
#           categories=["delta_q",
#                       "distribution_low", "distribution_high",
#                       "log_trial", "const"],
#           ordered=True))
#       .sort_values(["predictor", "model"])
#       .reset_index(drop=True))


In [33]:
fig, ax = plt.subplots(figsize=(4, 4))

palette = {'greedy': '#56B4E9', 'softmax': '#009E73', 'RS': '#065B8D'}
models = ['greedy', 'softmax', 'RS']
predictor_order = ['delta_q', 'dist_low', 'dist_high', 'log_trial', 'const']

# Prepare x-ticks
x_pos = np.arange(len(predictor_order))
width = 0.2  # space between models

# Plot each model
for i, model in enumerate(models):
    model_data = summary_all[summary_all['model'] == model]
    x_offsets = x_pos + (i - 1) * width  

    # Scatter plot
    ax.scatter(
        x_offsets,
        model_data['coef'],
        color=palette[model],
        s=20,
        alpha=0.8,
        edgecolors='none',
        label=model,
        zorder=3
    )

    # Error bars
    ax.errorbar(
        x_offsets,
        model_data['coef'],
        yerr=model_data['std_err (clustered)'],
        fmt='none',
        ecolor='black',
        capsize=4,        
        capthick=0.3,        
        elinewidth=0.3,     
        linewidth=0.3,   
        markersize=4
    )

    # Add * for significant p-values
    for x, y, pval in zip(x_offsets, model_data['coef'], model_data['p']):
        if pval < 0.05:
            ax.text(x, y + 0.4, '*', ha='center', va='bottom', fontsize=10, fontweight='bold')

# Axis styling
ax.axhline(0, linestyle='--', color='gray', linewidth=0.5)
ax.set_xticks(x_pos)
ax.set_ylim(-1, 7)
ax.set_xticklabels(predictor_order, fontsize=8)
ax.set_ylabel("logistic regression coefficient", fontsize=11)
ax.set_xlabel("predictor", fontsize=11)
ax.set_title("mixed-effects logistic regression coeffs", fontsize=11, fontweight='bold')
ax.legend(fontsize=9, title_fontsize=10, frameon=False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(0.5)
ax.spines['bottom'].set_linewidth(0.5)
plt.tight_layout()

# Save as PDF and SVG
# fig.savefig(output_dir / "deltaQ_mean_logistic_coeff.pdf")
# fig.savefig(output_dir / "deltaQ_mean_logistic_coeff.svg", format="svg", dpi=1200, bbox_inches="tight", transparent=True)


################################################################################################

# 1.  Sort by 'model' ----------------------------------------------
summary_sorted = (
    summary_all
    .sort_values('model')
    .reset_index(drop=True)
)

# ------------------------------------------------------------------
# 1-bis.  Format all numeric columns to 2-dp strings ---------------
summary_fmt = summary_sorted.copy()

# Move 'model' column to the front
cols = summary_fmt.columns.tolist()
cols.remove('model')
summary_fmt = summary_fmt[['model'] + cols]

num_cols = summary_fmt.select_dtypes(include='number').columns

# convert numbers to strings with six decimals; leave NaNs blank
for col in num_cols:
    summary_fmt[col] = summary_fmt[col].apply(
        lambda x: "" if pd.isna(x) else f"{x:.6f}"
    )
# ------------------------------------------------------------------
# 2.  Save as a PDF table (font size 8) -----------------------------


pdf_path = output_dir / "deltaQ_mean_logistic_coeff_table.pdf"

# Heuristic: 0.35 inch row height, 11 inch width
fig_height = 0.35 * len(summary_fmt) + 1
fig_width  = 10

with PdfPages(pdf_path) as pdf:
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))
    ax.axis('off')

    tbl = ax.table(
        cellText   = summary_fmt.values,
        colLabels  = summary_fmt.columns,
        cellLoc    = 'center',
        loc        = 'center'
    )

    # Style
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(8)

    for cell in tbl.get_celld().values():
        cell.set_edgecolor('black')
        cell.set_linewidth(0.25)

    # pdf.savefig(fig, bbox_inches='tight')
    # plt.close(fig)



## interpretation:
positive coefficient: higher odds of choosing arrow up

negative coefficient: higher odds of choosing arrow down

Positive coefficient for low and high: Compared to the uniform distribution (the reference), participants are more likely to choose 1 (e.g., "arrow up") when in the low distribution. Compared to uniform, participants are less likely to choose 1 when in the high distribution.

# delta q vals scatter plot